# Post-processing of case studies

## Usage notes

All figures will be written to the directory `img` as pdf and png files.

It will look for results in a "simulation/<simulator>" folder on the same level as this notebook. Simulators currently supported: "dymola", "optimica"

## Import required libraries

In [1]:
import os
import cases

import pandas as pd
import numpy as np

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from buildingspy.io.outputfile import Reader
from buildingspy.io.postprocess import Plotter

## Load in simulation results

In [2]:
# Helper methods

def get_results(case_name, simulator = "dymola"):
    """ Get the results for the case with name `case_name`
    from a "simulation/<simulator>" folder.

    Parameters:
    
        simulator: str
            "optimica" or "dymola"

    Returns:

        Reader: buildingspy.io.outputpy.Reader instance
    """
    
#     if simulator not in ["dymola", "optimica"]:
#         raise Exception("Unsupported simulator provided.")

    # First make sure that the simulation was successful

    # For results generated using Dymola
    if simulator == "dymola":

        dslog_name = os.path.join("simulations", simulator, case_name, "dslog.txt")
        with open(dslog_name) as dslog:
            if not "Integration terminated successfully" in dslog.read():
                raise Exception("Simulation failed. Check {}".format(dslog_name))

    # For results generated using Optimica
    elif simulator == "optimica":

        for file in os.listdir(os.path.join("simulations", simulator, case_name)):
            if '.json' in file:
                json_log_fullpath = os.path.join("simulations", simulator, case_name, file)
                import json
                if not json.load(open(json_log_fullpath))["translation"]["success"]:
                    raise Exception("Simulation failed. Check {}".format(json_log_fullpath))
    
    else:
        raise Exception("Simulation log found for neither supported simulation engine.")

    file_name = cases.get_result_fullpath(case_name, simulator)

    return Reader(file_name, simulator)


def get_partial_results(case_name, list_of_variables, simulator = "dymola", 
                        as_dataframe = False, equidistant = False):
    """ Get a dictionary with the variable names and the time series for `list_of_variables`

    Parameters:
    
        case_name: str
            Name of the simulated case

        list_of_variables: list
            Variable names for which to extract the result data

        simulator: str
            "optimica" or "dymola" 

        as_dataframe: bool
            If all variables have the same time record in the output one may convert the 
            resulting dictionary into a dataframe
            
        equidistant: bool
            Adds a column with the timestep size, if the results are with
            an equidistant timestep (no events).

    Returns:

        d: dataframe or dict
            holds time and varable for each variable listed in list_of_variables
    """
    reader = get_results(case_name, simulator = simulator)
    d = dict()
    for v in list_of_variables:
        try:
            d['time_' + v] = reader.values(v)[0]
            d[v] = reader.values(v)[1]
        except:
            print("Variable {} not found in case {} for simulator {}".format(v, case_name, simulator))

    if as_dataframe:
        import pandas as pd
        d = pd.DataFrame(d)
        if equidistant:
            d['ts_duration_min'] = pd.Series(
                d['time_' + list_of_variables[0]][1:].values - d[
                    'time_' + list_of_variables[0]][:-1].values).unique()[0]/60

    return d

In [3]:
# Read in all simulation results

all_case_names = cases.get_list_of_case_names()
print("Found cases:", all_case_names)

# res = {"optimica" : dict(), "dymola" : dict()}
# simulators = ["dymola", "optimica"]

res = {"dymola" : dict()}
simulators = ["dymola"]

for simulator in simulators:
    for case_name in all_case_names:
        try:
            res[simulator][case_name] = get_results(case_name, simulator)
        except:
            print("Warning: Could not read results, please check simulation log file: ", case_name)

print("Read in simulation results.")

Found cases: ['base_summer', 'alt_summer', 'base_annual', 'alt_annual', 'base_summer_plus_two', 'alt_summer_plus_two', 'base_annual_plus_two', 'alt_annual_plus_two', 'base_summer_less_two', 'alt_summer_less_two', 'base_annual_less_two', 'alt_annual_less_two', 'base_wse_limit', 'alt_wse_limit', 'base_below_wse_limit', 'alt_below_wse_limit']
Read in simulation results.


## Explore variables found in the simulation results

In [4]:
# aggregate data timeseries

PAgg = {"dymola": dict()}#, "optimica": dict()}
simulators = ["dymola"]#, "optimica"]
for simulator in simulators:
    for case in all_case_names:
        PAgg[simulator][case] = get_partial_results(
            case,  ["PConWatPumAgg.y", "PWSEWatPumAgg.y", "PCooTowWatPumAgg.y", "PChiWatPumAgg.y", "PChiAgg.y", "PCooTowAgg.y", "PSupFanAgg.y","QRooIntGaiAgg.y"], 
            simulator, as_dataframe=True, equidistant = simulator == "dymola")
        PAgg[simulator][case] = PAgg[simulator][case].loc[(PAgg[simulator][case].index % 2)==0,:].reset_index(drop=True)

Variable PWSEWatPumAgg.y not found in case alt_summer for simulator dymola
Variable PCooTowWatPumAgg.y not found in case alt_summer for simulator dymola
Variable PWSEWatPumAgg.y not found in case alt_annual for simulator dymola
Variable PCooTowWatPumAgg.y not found in case alt_annual for simulator dymola
Variable PWSEWatPumAgg.y not found in case alt_summer_plus_two for simulator dymola
Variable PCooTowWatPumAgg.y not found in case alt_summer_plus_two for simulator dymola
Variable PWSEWatPumAgg.y not found in case alt_annual_plus_two for simulator dymola
Variable PCooTowWatPumAgg.y not found in case alt_annual_plus_two for simulator dymola
Variable PWSEWatPumAgg.y not found in case alt_summer_less_two for simulator dymola
Variable PCooTowWatPumAgg.y not found in case alt_summer_less_two for simulator dymola
Variable PWSEWatPumAgg.y not found in case alt_annual_less_two for simulator dymola
Variable PCooTowWatPumAgg.y not found in case alt_annual_less_two for simulator dymola
Variable P

In [5]:
PAgg["dymola"]['alt_annual'].head(20)

,time_PConWatPumAgg.y,PConWatPumAgg.y,time_PChiWatPumAgg.y,PChiWatPumAgg.y,time_PChiAgg.y,PChiAgg.y,time_PCooTowAgg.y,PCooTowAgg.y,time_PSupFanAgg.y,PSupFanAgg.y,time_QRooIntGaiAgg.y,QRooIntGaiAgg.y,ts_duration_min
0,0.0,0.000000e+00,0.0,0.00,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,5.0
1,600.0,0.000000e+00,600.0,0.00,600.0,0.00000,600.0,0.000000,600.0,6782264.0,600.0,2500000.0,5.0
2,1200.0,0.000000e+00,1200.0,0.00,1200.0,0.00000,1200.0,0.000000,1200.0,13564528.0,1200.0,10000000.0,5.0
3,1800.0,0.000000e+00,1800.0,0.00,1800.0,0.00000,1800.0,0.000000,1800.0,20346792.0,1800.0,22500000.0,5.0
4,2400.0,4.529710e+03,2400.0,2025882.00,2400.0,198388.34375,2400.0,0.000000,2400.0,27129056.0,2400.0,40000000.0,5.0
5,3000.0,1.173785e+06,3000.0,2260837.00,3000.0,198388.34375,3000.0,31534.576172,3000.0,33911320.0,3000.0,62500000.0,5.0
6,3600.0,2.343041e+06,3600.0,2364535.25,3600.0,198388.34375,3600.0,31534.576172,3600.0,40693584.0,3600.0,90000000.0,5.0
7,4200.0,3.512297e+06,4200.0,2468233.75,4200.0,198388.34375,4200.0,31534.576172,4200.0,47475848.0,4200.0,122500000.0,5.0
8,4800.0,4.681552e+06,4800.0,2571932.00,4800.0,198388.34375,4800.0,31534.576172,4800.0,54258112.0,4800.0,160000000.0,5.0
9,5400.0,5.850808e+06,5400.0,2675630.50,5400.0,198388.34375,5400.0,31534.576172,5400.0,61040376.0,5400.0,202500000.0,5.0


In [6]:
data = {"dymola": dict()}#, "optimica": dict()}
simulators = ["dymola"]#, "optimica"]

for simulator in simulators:
    
    for case in all_case_names:
        
        var_labels = ["weaBus.TWetBul", "pumCW.m_flow", "chilledWaterReset.uChiWatPlaRes","chilledWaterReset.TChiWatSupSet",
                      "chilledWaterReset.dpChiWatPumSet", "pumCW.VMachine_flow", "pumCW.eta", "pumCW.P", "TAirSup.T", "chi.on", 
                      "cooTow.y", "pumCW.m_flow", "chi.m1_flow"]
            
        var_labels += ["chi.PLR2", "chi.PLR1", "chi.P", "chi.TEvaLvg", "chi.TEvaEnt", "chi.TConLvg", "chi.TConEnt"]
        
        if 'base' in case:
            var_labels += ["pumCWWSE.m_flow", "pumCT.m_flow", "waterSideEconomizerOnOff.yOn", "condenserWater.heaPreCon.y" ,"val.y_actual","val.y",]
            
        elif 'alt' in case:
            var_labels += ["heaPreCon.yHeaPreConVal", "yWSEOn.y","heaPreCon.chiHeaPreLoo.conPID.y","towCon.towFanSpe.fanSpeWse.wseOpe.chiWatTemCon.y", "towCon.towFanSpe.fanSpeWse.intOpe.loaCon.y",
                           "staSetCon.staUp.hysTSup.u","staSetCon.staDow.hysTSup.u",]
            
        if "_wse_limit" in case:
            var_labels += ["TwtBulDes.y"]
            
        data[simulator][case] = get_partial_results(
            case,  var_labels, simulator, as_dataframe = True,equidistant = simulator == "dymola")
        
        data[simulator][case] = data[simulator][case].loc[(data[simulator][case].index % 2)==0,:].reset_index(drop = True)
        data[simulator][case] = data[simulator][case][data[simulator][case].columns.drop(list(data[simulator][case].filter(regex='time_')))]


In [7]:
# check sensitivity variables

print(get_partial_results("alt_annual_plus_two", ['chilledWaterReset.TChiWatSupMax'], 'dymola'))
print(get_partial_results("alt_annual_less_two", ['chilledWaterReset.TChiWatSupMax'], 'dymola'))
print(get_partial_results("alt_annual", ['chilledWaterReset.TChiWatSupMax'], 'dymola'))

print(get_partial_results("base_annual_plus_two", ['chilledWaterReset.linPieTwo.y20'], 'dymola'))
print(get_partial_results("base_annual_less_two", ['chilledWaterReset.linPieTwo.y20'], 'dymola'))
print(get_partial_results("base_annual", ['chilledWaterReset.linPieTwo.y20'], 'dymola'))

print(get_partial_results("alt_annual_plus_two", ['TZonSupSet'], 'dymola'))
print(get_partial_results("alt_annual_less_two", ['TZonSupSet'], 'dymola'))
print(get_partial_results("alt_annual", ['TZonSupSet'], 'dymola'))

print(get_partial_results("base_annual_plus_two", ['TZonSupSet'], 'dymola'))
print(get_partial_results("base_annual_less_two", ['TZonSupSet'], 'dymola'))
print(get_partial_results("base_annual", ['TZonSupSet'], 'dymola'))

{'time_chilledWaterReset.TChiWatSupMax': array([       0., 31536000.], dtype=float32), 'chilledWaterReset.TChiWatSupMax': array([297.15, 297.15], dtype=float32)}
{'time_chilledWaterReset.TChiWatSupMax': array([       0., 31536000.], dtype=float32), 'chilledWaterReset.TChiWatSupMax': array([293.15, 293.15], dtype=float32)}
{'time_chilledWaterReset.TChiWatSupMax': array([       0., 31536000.], dtype=float32), 'chilledWaterReset.TChiWatSupMax': array([295.15, 295.15], dtype=float32)}
{'time_chilledWaterReset.linPieTwo.y20': array([       0., 31536000.], dtype=float32), 'chilledWaterReset.linPieTwo.y20': array([297.15, 297.15], dtype=float32)}
{'time_chilledWaterReset.linPieTwo.y20': array([       0., 31536000.], dtype=float32), 'chilledWaterReset.linPieTwo.y20': array([293.15, 293.15], dtype=float32)}
{'time_chilledWaterReset.linPieTwo.y20': array([       0., 31536000.], dtype=float32), 'chilledWaterReset.linPieTwo.y20': array([295.15, 295.15], dtype=float32)}
{'time_TZonSupSet': array([ 

In [8]:
all_case_names[:-2]

['base_summer',
 'alt_summer',
 'base_annual',
 'alt_annual',
 'base_summer_plus_two',
 'alt_summer_plus_two',
 'base_annual_plus_two',
 'alt_annual_plus_two',
 'base_summer_less_two',
 'alt_summer_less_two',
 'base_annual_less_two',
 'alt_annual_less_two',
 'base_wse_limit',
 'alt_wse_limit']

In [9]:
print(data["dymola"]["alt_annual"].loc[:,"cooTow.y"].describe().round(2))

count    52561.00
mean         0.39
std          0.20
min          0.00
25%          0.29
50%          0.29
75%          0.38
max          1.00
Name: cooTow.y, dtype: float64


In [10]:
data["dymola"]["base_annual"]["condenserWater.heaPreCon.y"].unique()

array([1., 0.], dtype=float32)

In [11]:
print(data["dymola"]["base_annual"].describe().round(2))

       weaBus.TWetBul  pumCW.m_flow  chilledWaterReset.uChiWatPlaRes  \
count        52561.00      52561.00                         52561.00   
mean           284.30          0.03                             0.02   
std              4.60          0.75                             0.04   
min            271.43          0.00                             0.00   
25%            280.84          0.00                             0.00   
50%            284.48          0.00                             0.01   
75%            287.54          0.00                             0.03   
max            296.59         19.84                             0.31   

       chilledWaterReset.TChiWatSupSet  chilledWaterReset.dpChiWatPumSet  \
count                         52561.00                          52561.00   
mean                            295.15                          18379.06   
std                               0.00                           9224.96   
min                             295.15         

### Data exploration

In [12]:
data["dymola"]["alt_annual"].loc[(data["dymola"]["alt_annual"]["cooTow.y"] > 0)&(data["dymola"]["alt_annual"]["chi.on"] > 0.),:]

,weaBus.TWetBul,pumCW.m_flow,chilledWaterReset.uChiWatPlaRes,chilledWaterReset.TChiWatSupSet,chilledWaterReset.dpChiWatPumSet,pumCW.VMachine_flow,pumCW.eta,pumCW.P,TAirSup.T,chi.on,...,chi.TConLvg,chi.TConEnt,heaPreCon.yHeaPreConVal,yWSEOn.y,heaPreCon.chiHeaPreLoo.conPID.y,towCon.towFanSpe.fanSpeWse.wseOpe.chiWatTemCon.y,towCon.towFanSpe.fanSpeWse.intOpe.loaCon.y,staSetCon.staUp.hysTSup.u,staSetCon.staDow.hysTSup.u,ts_duration_min
26719,295.148163,39.682541,0.175651,295.149994,53977.527344,0.039858,0.49,7806.691895,300.068817,1.0,...,299.499512,296.436920,0.200000,1.0,1.000000,0.791525,0.000000,0.000237,-5.855677,5.0
26720,295.240845,39.682541,0.172824,295.149994,53317.593750,0.039858,0.49,7806.691895,300.149109,1.0,...,299.763977,296.580536,0.200000,1.0,1.000000,0.792745,0.973126,0.000246,-5.932322,5.0
26721,295.337250,39.682541,0.172982,295.149994,53354.402344,0.039858,0.49,7806.691895,300.150146,1.0,...,299.844879,296.624939,0.200000,1.0,1.000000,0.793986,1.000000,0.000249,-6.030255,5.0
26722,295.432373,39.682541,0.172997,295.149994,53357.808594,0.039858,0.49,7806.691895,300.150024,1.0,...,300.003296,296.714844,0.200000,1.0,1.000000,0.795249,1.000000,0.000254,-6.123427,5.0
26723,295.521820,39.682541,0.172997,295.149994,53357.945312,0.039858,0.49,7791.691406,300.149994,1.0,...,300.153534,296.800140,0.200916,1.0,0.998855,0.796538,1.000000,0.000259,-6.213765,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33848,287.420685,39.682541,0.172996,295.149994,53357.750000,0.039858,0.49,7806.691895,300.149994,1.0,...,299.770966,296.577087,0.200000,1.0,1.000000,0.909549,0.112779,0.000246,1.881662,5.0
33849,287.387634,39.682541,0.172996,295.149994,53357.750000,0.039858,0.49,7806.691895,300.149994,1.0,...,299.770996,296.577118,0.200000,1.0,1.000000,0.910780,0.112629,0.000246,1.929363,5.0
33850,287.355377,39.682541,0.172996,295.149994,53357.750000,0.039858,0.49,7806.691895,300.149994,1.0,...,299.770996,296.577118,0.200000,1.0,1.000000,0.912012,0.112483,0.000246,1.947235,5.0
33851,287.319153,39.682541,0.172996,295.149994,53357.750000,0.039858,0.49,7806.691895,300.149994,1.0,...,299.770996,296.577087,0.200000,1.0,1.000000,0.913243,0.112322,0.000246,1.983235,5.0


In [13]:
# two curves looking at the tower_fan_speed_vs_wetbul
aa = data["dymola"]["alt_annual"].loc[(data["dymola"]["alt_annual"]["cooTow.y"] < 0.8)&(data["dymola"]["alt_annual"]["cooTow.y"] > 0.6)&(
    data["dymola"]["alt_annual"]["weaBus.TWetBul"] < (273.15+20)),:]
aa["chi.on"].mean()

0.0

In [14]:
# data["dymola"]["base_annual"].loc[(data["dymola"]["base_annual"]["val.y"] < 0.9)&(data["dymola"]["base_annual"]["chi.on"]),:] 
data["dymola"]["base_annual"].loc[data["dymola"]["base_annual"]["condenserWater.heaPreCon.y"] > 0.,["chi.PLR2", "chi.PLR1", "chi.P", "chi.TEvaLvg", "chi.TEvaEnt", "chi.TConLvg", "chi.TConEnt", "condenserWater.heaPreCon.y" ,"val.y","chi.on"]] 

,chi.PLR2,chi.PLR1,chi.P,chi.TEvaLvg,chi.TEvaEnt,chi.TConLvg,chi.TConEnt,condenserWater.heaPreCon.y,val.y,chi.on
0,0.0,0.0,0.0,293.149994,293.149994,298.149994,293.149994,1.0,0.0,0.0
1,0.0,0.0,0.0,293.150055,294.492523,298.149994,293.149994,1.0,0.0,0.0
2,0.0,0.0,0.0,293.150330,293.842224,298.149994,293.149994,1.0,0.0,0.0
3,0.0,0.0,0.0,293.149445,277.841827,298.149994,293.149994,1.0,0.0,0.0
4,0.0,0.0,0.0,293.148285,276.803040,298.149994,293.149994,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
52556,0.0,0.0,0.0,290.328522,286.858582,293.880249,295.846832,1.0,0.0,0.0
52557,0.0,0.0,0.0,290.328247,286.890961,293.880249,295.846832,1.0,0.0,0.0
52558,0.0,0.0,0.0,290.328003,286.940796,293.880249,295.846832,1.0,0.0,0.0
52559,0.0,0.0,0.0,290.327759,287.005615,293.880249,295.846863,1.0,0.0,0.0


In [15]:
# aggregate data timeseries

data_df = {"dymola": dict()}
simulators = ["dymola"]
for simulator in simulators:
    for case in all_case_names:
        data_df[simulator][case] = get_partial_results(
            case,  ["chi.COP", "chi.TSet", "chilledWaterReset.TChiWatSupSet", "chilledWaterReset.uChiWatPlaRes", "chi.TEvaLvg", "chi.TEvaEnt", "chi.TConLvg", "chi.TConEnt", "weaBus.TWetBul", "cooTow.y", "TCHWEntChi.T"], 
            simulator, as_dataframe=True, equidistant = simulator == "dymola")
        
        data_df[simulator][case] = data_df[simulator][case].loc[(data_df[simulator][case].index % 2)==0,:].reset_index(drop=True)
        data_df["dymola"][case] = data_df["dymola"][case][data_df["dymola"][case].columns.drop(list(data_df["dymola"][case].filter(regex='time_')))]

In [16]:
print(data_df["dymola"]["base_annual"].describe().round(2))
# data_df["dymola"]["base_annual"].loc[(data_df["dymola"]["base_annual"]["chilledWaterReset.TChiWatSupSet"] > 288) & (data_df["dymola"]["base_annual"]["chilledWaterReset.TChiWatSupSet"] < 291),:]
print((data_df["dymola"]["base_annual"]["chilledWaterReset.uChiWatPlaRes"] > 0.8).sum())
print((data_df["dymola"]["base_annual"]["chilledWaterReset.uChiWatPlaRes"] <= 0.8).sum())
data_df["dymola"]["base_annual"].shape

        chi.COP  chi.TSet  chilledWaterReset.TChiWatSupSet  \
count  52561.00  52561.00                         52561.00   
mean       0.01    295.15                           295.15   
std        0.37      0.00                             0.00   
min        0.00    295.15                           295.15   
25%        0.00    295.15                           295.15   
50%        0.00    295.15                           295.15   
75%        0.00    295.15                           295.15   
max       10.65    295.15                           295.15   

       chilledWaterReset.uChiWatPlaRes  chi.TEvaLvg  chi.TEvaEnt  chi.TConLvg  \
count                         52561.00     52561.00     52561.00     52561.00   
mean                              0.02       292.02       289.94       296.06   
std                               0.04         1.93         3.36         2.14   
min                               0.00       289.46       276.67       292.81   
25%                               0.

(52561, 12)

In [17]:
296.18 - 273.15

23.03000000000003

In [18]:
print("Chiller on fraction, alternative: ", 
      data["dymola"]["alt_annual"].groupby(["chi.on"]).count().reset_index().iloc[1,1]/data["dymola"]["alt_annual"].groupby(["chi.on"]).count().reset_index().iloc[0,1])
print("Chiller on fraction, base: ", 
      data["dymola"]["base_annual"].groupby(["chi.on"]).count().reset_index().iloc[1,1]/data["dymola"]["alt_annual"].groupby(["chi.on"]).count().reset_index().iloc[0,1])

Chiller on fraction, alternative:  0.0029002652216222405
Chiller on fraction, base:  0.0014501326108111202


In [19]:
data["dymola"]["alt_annual"].loc[data["dymola"]["alt_annual"]["chi.on"] == 1.,["cooTow.y","weaBus.TWetBul"]]

,cooTow.y,weaBus.TWetBul
26719,1.000000,295.148163
26720,0.978501,295.240845
26721,1.000000,295.337250
26722,1.000000,295.432373
26723,1.000000,295.521820
...,...,...
33848,0.290223,287.420685
33849,0.290103,287.387634
33850,0.289986,287.355377
33851,0.289858,287.319153


In [20]:
data["dymola"]["base_annual"].loc[data["dymola"]["base_annual"]["chi.on"] == 1.,["cooTow.y","weaBus.TWetBul"]]

,cooTow.y,weaBus.TWetBul
26720,1.0,295.240845
26721,1.0,295.337250
26722,1.0,295.432373
26723,1.0,295.521820
26724,1.0,295.601532
...,...,...
33811,1.0,292.643707
33812,1.0,292.316650
33813,1.0,291.976654
33814,1.0,291.638855


In [21]:
# print(data["dymola"]["alt_all_annual"].describe().round(2))

chi_on_alt = data["dymola"]["alt_annual"][data["dymola"]["alt_annual"]["chi.on"]>0]

chi_on_base = data["dymola"]["base_annual"][data["dymola"]["base_annual"]["chi.on"]>0]

print(chi_on_base.describe().round(2))

       weaBus.TWetBul  pumCW.m_flow  chilledWaterReset.uChiWatPlaRes  \
count           76.00         76.00                            76.00   
mean           294.48         19.84                             0.17   
std              1.55          0.00                             0.00   
min            291.18         19.84                             0.17   
25%            293.48         19.84                             0.17   
50%            294.80         19.84                             0.17   
75%            295.86         19.84                             0.17   
max            296.59         19.84                             0.18   

       chilledWaterReset.TChiWatSupSet  chilledWaterReset.dpChiWatPumSet  \
count                            76.00                             76.00   
mean                            295.15                          53396.40   
std                               0.00                            256.26   
min                             295.15         

In [22]:
# fig = plt.figure(figsize=(4.5, 4.5))
# ax = fig.add_subplot(1, 1, 1)

# ax.scatter(chi_on_alt["cooTow.y"], chi_on_alt["weaBus.TWetBul"], linewidth=0.5, s = 1.)

# save_plot(fig, "test")

In [23]:
int(len(chi_on_base)*0.1)

7

In [24]:
print(chi_on_alt.describe().round(2))

       weaBus.TWetBul  pumCW.m_flow  chilledWaterReset.uChiWatPlaRes  \
count          152.00        152.00                           152.00   
mean           291.68         39.68                             0.17   
std              3.18          0.00                             0.00   
min            287.27         39.68                             0.17   
25%            288.60         39.68                             0.17   
50%            291.35         39.68                             0.17   
75%            295.02         39.68                             0.17   
max            296.59         39.68                             0.19   

       chilledWaterReset.TChiWatSupSet  chilledWaterReset.dpChiWatPumSet  \
count                           152.00                            152.00   
mean                            295.15                          53385.35   
std                               0.00                            300.72   
min                             295.15         

### Some statistics

In [25]:
# to add months to equidistant data

df = data["dymola"]['base_annual']
df['12 bins'] = list(df.index/(data["dymola"]['base_annual'].shape[0]/12))
df['12 bins'] = np.ceil(df['12 bins'])

count_wse_on_per_month = df.groupby('12 bins').sum()["waterSideEconomizerOnOff.yOn"].reset_index()

# df.index

# list(df.index/12)

# base_annual = df[df["chi.on"] != 1.][df["waterSideEconomizerOnOff.yOn"] == 1.]
df1 = data["dymola"]['alt_annual']
# wse_annual = df1[df1["chi.on"] != 1.][df1["yWSEOn.y"] == 1.]
# df2 = data["dymola"]['alt_heapreval_wse_annual']
# cwreset_wse_annual = df2[df2["chi.on"] != 1.][df2["yWSEOn.y"] == 1.]

# print(base_annual.shape)
# print(wse_annual.shape)
# print(cwreset_wse_annual.shape)



In [26]:
pd.options.display.float_format = '{:,.2f}'.format

data1 = data.copy()

for case in all_case_names:
    data1["dymola"][case] = data["dymola"][case][data["dymola"][case].columns.drop(list(data["dymola"][case].filter(regex='time_')))]

for case in all_case_names:
    print("\n {case_name}, simulation averages: \n".format(case_name = case))

    print(data1["dymola"][case].mean())
    print("\nChiller ON % of time", data1["dymola"][case].mean()["chi.on"])
    try:
        print("WSE ON % of time", data1["dymola"][case].mean()["waterSideEconomizerOnOff.yOn"])
        print("Data records where WSE is on and chiller is not:", data1["dymola"][case].loc[
            :, "waterSideEconomizerOnOff.yOn"].sum() - sum(data1["dymola"][case].loc[
            :, "waterSideEconomizerOnOff.yOn"] == data1["dymola"][case].loc[:, "chi.on"]))
    except:
        print("WSE ON % of time", data1["dymola"][case].mean()["yWSEOn.y"])
        print("Data records where WSE is on and chiller is not:", data1["dymola"][case].loc[
            :, "yWSEOn.y"].sum() - sum(data1["dymola"][case].loc[
            :, "yWSEOn.y"] == data1["dymola"][case].loc[:, "chi.on"]))


 base_summer, simulation averages: 

weaBus.TWetBul                        288.76
pumCW.m_flow                            0.11
chilledWaterReset.uChiWatPlaRes         0.06
chilledWaterReset.TChiWatSupSet       295.15
chilledWaterReset.dpChiWatPumSet   26,896.41
pumCW.VMachine_flow                     0.00
pumCW.eta                               0.49
pumCW.P                                11.24
TAirSup.T                             300.14
chi.on                                  0.01
cooTow.y                                1.00
chi.m1_flow                             0.11
chi.PLR2                                0.00
chi.PLR1                                0.00
chi.P                                  68.16
chi.TEvaLvg                           294.40
chi.TEvaEnt                           293.13
chi.TConLvg                           293.33
chi.TConEnt                           294.79
pumCWWSE.m_flow                        19.84
pumCT.m_flow                           19.95
waterSideEconomiz

In [27]:
# data1["dymola"]["alt_all_annual"][data1["dymola"]["alt_all_annual"]["cooTow.y"]<=0.2]
# data1["dymola"]["base_annual"][data1["dymola"]["base_annual"]["cooTow.y"]==0.]

# Compile total energy use and PUE per device, case, and simulator

## Energy use

We obtain the energy use as cumulative power consumed by:

- Zone supply fan: fan.P
- Chilled water pump: pumCHW.P
- Condenser water pump: pumCW.P
- Chiller: chi.P
- Cooling tower: cooTow.PFan

We also look at the aggregate IT Heat Load

In [28]:
from pdb import set_trace as bp

vlmap = {
    "PConWatPumAgg.y" : "CW Pump",
    "PWSEWatPumAgg.y" : "WSE Pump",
    "PCooTowWatPumAgg.y" : "CT Pump",  
    "PChiWatPumAgg.y" : "CHW Pump",
    "PChiAgg.y" : "Chiller",
    "PCooTowAgg.y" : "Cooling Tower Fan",
    "PSupFanAgg.y" : "Supply Fan",
    "QRooIntGaiAgg.y" : "Data Center Zone Load"
}


# Season: use value from the case dist

# Controller design: use long name of the case dict

main_cases = cases.get_cases()[:4]
sensitivity_cases_plus_2 = cases.get_cases()[4:8]
sensitivity_cases_less_2 = cases.get_cases()[8:12]

# PAgg_compiled = pd.DataFrame(columns = [["Simulator", "Controller Design", "Season"] + list(vlmap.values())])
PAgg_compiled = dict()


for sensitivity in ['+2', '-2', '']:
    inx = 0

    if sensitivity == '+2':
        _cases = sensitivity_cases_plus_2
        sens_key = '_sens+2'
    elif sensitivity == '-2':
        _cases = sensitivity_cases_less_2
        sens_key = '_sens-2'
    else:
        _cases = main_cases
        sens_key = ''
    
    try:
        del PAgg_compiled[sens_key]
    except:
        pass

    PAgg_compiled[sens_key] = pd.DataFrame(columns = [["Simulator", "Controller Design", "Season"] + list(vlmap.values())][0])

    for simulator in simulators:
        for case in _cases:
 
            PAgg_compiled[sens_key].loc[inx, "Simulator"] = simulator

            PAgg_compiled[sens_key].loc[inx, "Controller Design"] = [i for i in ["Base-case", "Alternative"] if i in case['long_name']][0]
            PAgg_compiled[sens_key].loc[inx, "Season"] = case['season']
            for var in list(vlmap.keys()):
                # grab the last value in the cumulative result variables and convert to kWh from Ws
                try:
                    PAgg_compiled[sens_key].loc[inx, vlmap[var]] = (PAgg[simulator][case['name']].loc[:,var][-1:].values[0]/(1000*3600)).round(1)
                except:
                    PAgg_compiled[sens_key].loc[inx, vlmap[var]] = np.nan

            inx += 1

    # e.g. lights, people, UPSs
    fraction_IT_load = .8
    PAgg_compiled[sens_key]["Non-IT Heat Load"] = PAgg_compiled[sens_key]["Data Center Zone Load"] * (1 - fraction_IT_load)
    PAgg_compiled[sens_key]["IT Heat Load"] = PAgg_compiled[sens_key]["Data Center Zone Load"] * fraction_IT_load

    PAgg_compiled[sens_key] = PAgg_compiled[sens_key].drop(columns = ["Data Center Zone Load"])
    PAgg_compiled[sens_key]["Total Energy Use [kWh]"] = PAgg_compiled[sens_key].loc[:,vlmap['PConWatPumAgg.y'] : vlmap["PSupFanAgg.y"]].sum(axis = 1)
    PAgg_compiled[sens_key]["Total Energy Use incl. non-IT [kWh]"] = PAgg_compiled[sens_key].loc[:,vlmap['PConWatPumAgg.y'] : "Non-IT Heat Load"].sum(axis = 1)

PAgg_compiled[sens_key]

,Simulator,Controller Design,Season,CW Pump,WSE Pump,CT Pump,CHW Pump,Chiller,Cooling Tower Fan,Supply Fan,Non-IT Heat Load,IT Heat Load,Total Energy Use [kWh],Total Energy Use incl. non-IT [kWh]
0,dymola,Base-case,summer,26.00,"4,003.20",356.90,"1,269.00",155.90,"14,819.90","25,772.60","228,000.00","912,000.00","46,403.50","274,403.50"
1,dymola,Alternative,summer,"4,583.60",NaN,NaN,"2,151.00",423.90,"4,834.40","25,772.60","228,000.00","912,000.00","37,765.50","265,765.50"
2,dymola,Base-case,annual,25.20,"15,382.00","1,333.10","2,760.10",155.80,"56,937.80","99,021.10","875,500.02","3,502,000.08","175,615.10","1,051,115.12"
3,dymola,Alternative,annual,"17,210.70",NaN,NaN,"7,582.30",422.60,"6,878.30","99,021.10","875,500.08","3,502,000.32","131,115.00","1,006,615.08"


In [29]:
PAgg_compiled['_sens+2']

,Simulator,Controller Design,Season,CW Pump,WSE Pump,CT Pump,CHW Pump,Chiller,Cooling Tower Fan,Supply Fan,Non-IT Heat Load,IT Heat Load,Total Energy Use [kWh],Total Energy Use incl. non-IT [kWh]
0,dymola,Base-case,summer,0.70,"4,003.10",343.60,772.90,0.10,"14,819.80","25,772.60","228,000.00","912,000.00","45,712.80","273,712.80"
1,dymola,Alternative,summer,"4,442.70",NaN,NaN,"1,925.70",0.80,"1,886.10","25,772.60","228,000.00","912,000.00","34,027.90","262,027.90"
2,dymola,Base-case,annual,0.00,"15,381.80","1,319.80","2,021.50",0.00,"56,937.80","99,021.10","875,500.02","3,502,000.08","174,682.00","1,050,182.02"
3,dymola,Alternative,annual,"17,069.80",NaN,NaN,"7,346.30",0.00,"2,630.60","99,021.10","875,500.08","3,502,000.32","126,067.80","1,001,567.88"


In [30]:
PAgg_compiled['_sens-2']

,Simulator,Controller Design,Season,CW Pump,WSE Pump,CT Pump,CHW Pump,Chiller,Cooling Tower Fan,Supply Fan,Non-IT Heat Load,IT Heat Load,Total Energy Use [kWh],Total Energy Use incl. non-IT [kWh]
0,dymola,Base-case,summer,795.50,"3,999.00",777.40,"1,904.10","5,597.90","14,819.90","25,772.60","228,000.00","912,000.00","53,666.40","281,666.40"
1,dymola,Alternative,summer,"11,109.60",NaN,NaN,"2,569.50","21,146.20","5,437.60","25,772.60","228,000.00","912,000.00","66,035.50","294,035.50"
2,dymola,Base-case,annual,801.40,"15,377.60","1,757.30","3,965.30","5,627.30","56,938.40","99,021.10","875,500.02","3,502,000.08","183,488.40","1,058,988.42"
3,dymola,Alternative,annual,"24,095.30",NaN,NaN,"8,169.10","22,210.40","10,338.00","99,021.10","875,500.14","3,502,000.56","163,833.90","1,039,334.04"


In [31]:
for sensitivity in ['', '+2', '-2']:
    
    if sensitivity!='':    
        print("\n Sensitivity with TZonSet of 27" + sensitivity + " degC")
    else:
        print("\n TZonSet is 27 degC")
    
    if sensitivity == '+2':
        _cases = sensitivity_cases_plus_2
        sens_key = '_sens+2'
    elif sensitivity == '-2':
        _cases = sensitivity_cases_less_2
        sens_key = '_sens-2'
    else:
        _cases = main_cases
        sens_key = ''
    
    # annual savings/gains
    print("Total savings:")
    print("Annual savings in alternative case" + sens_key + ": ", 
          (PAgg_compiled[sens_key].loc[2,"Total Energy Use [kWh]"]-PAgg_compiled[sens_key].loc[3,"Total Energy Use [kWh]"])/PAgg_compiled[sens_key].loc[2,"Total Energy Use [kWh]"])
    print("Seasonal savings in alternative case" + sens_key + ": ",
          (PAgg_compiled[sens_key].loc[0,"Total Energy Use [kWh]"]-PAgg_compiled[sens_key].loc[1,"Total Energy Use [kWh]"])/PAgg_compiled[sens_key].loc[0,"Total Energy Use [kWh]"])

    # tower fan savings/gains
    print("Tower fan savings:")
    print("Annual savings in alternative case" + sens_key + ": ", 
          (PAgg_compiled[sens_key].loc[2,"Cooling Tower Fan"]-PAgg_compiled[sens_key].loc[3,"Cooling Tower Fan"])/PAgg_compiled[sens_key].loc[2,"Cooling Tower Fan"])
    print("Seasonal savings in alternative case" + sens_key + ": ",
          (PAgg_compiled[sens_key].loc[0,"Cooling Tower Fan"]-PAgg_compiled[sens_key].loc[1,"Cooling Tower Fan"])/PAgg_compiled[sens_key].loc[0,"Cooling Tower Fan"])

    # chw pump savings/gains
    print("CHW pump savings:")
    print("Annual savings in alternative case" + sens_key + ": ", 
          (PAgg_compiled[sens_key].loc[2,"CHW Pump"]-PAgg_compiled[sens_key].loc[3,"CHW Pump"])/PAgg_compiled[sens_key].loc[2,"CHW Pump"])
    print("Seasonal savings in alternative case" + sens_key + ": ",
          (PAgg_compiled[sens_key].loc[0,"CHW Pump"]-PAgg_compiled[sens_key].loc[1,"CHW Pump"])/PAgg_compiled[sens_key].loc[0,"CHW Pump"])

    # chiller savings/gains
    print("Chiller savings:")
    print("Annual savings in alternative case" + sens_key + ": ", 
          (PAgg_compiled[sens_key].loc[2,"Chiller"]-PAgg_compiled[sens_key].loc[3,"Chiller"])/PAgg_compiled[sens_key].loc[2,"Chiller"])
    print("Seasonal savings in alternative case" + sens_key + ": ",
          (PAgg_compiled[sens_key].loc[0,"Chiller"]-PAgg_compiled[sens_key].loc[1,"Chiller"])/PAgg_compiled[sens_key].loc[0,"Chiller"])


 TZonSet is 27 degC
Total savings:
Annual savings in alternative case:  0.2533956362522357
Seasonal savings in alternative case:  0.18614975163511374
Tower fan savings:
Annual savings in alternative case:  0.8791962457277941
Seasonal savings in alternative case:  0.673789971592251
CHW pump savings:
Annual savings in alternative case:  -1.7471106119343505
Seasonal savings in alternative case:  -0.6950354609929078
Chiller savings:
Annual savings in alternative case:  -1.712451861360719
Seasonal savings in alternative case:  -1.7190506735086593

 Sensitivity with TZonSet of 27+2 degC
Total savings:
Annual savings in alternative case_sens+2:  0.27830114150284513
Seasonal savings in alternative case_sens+2:  0.25561549500358766
Tower fan savings:
Annual savings in alternative case_sens+2:  0.9537987066588453
Seasonal savings in alternative case_sens+2:  0.8727310759929283
CHW pump savings:
Annual savings in alternative case_sens+2:  -2.6340836012861737
Seasonal savings in alternative case_

/home/milicag/venvs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars


In [32]:
PAgg_compiled[sens_key]

,Simulator,Controller Design,Season,CW Pump,WSE Pump,CT Pump,CHW Pump,Chiller,Cooling Tower Fan,Supply Fan,Non-IT Heat Load,IT Heat Load,Total Energy Use [kWh],Total Energy Use incl. non-IT [kWh]
0,dymola,Base-case,summer,795.50,"3,999.00",777.40,"1,904.10","5,597.90","14,819.90","25,772.60","228,000.00","912,000.00","53,666.40","281,666.40"
1,dymola,Alternative,summer,"11,109.60",NaN,NaN,"2,569.50","21,146.20","5,437.60","25,772.60","228,000.00","912,000.00","66,035.50","294,035.50"
2,dymola,Base-case,annual,801.40,"15,377.60","1,757.30","3,965.30","5,627.30","56,938.40","99,021.10","875,500.02","3,502,000.08","183,488.40","1,058,988.42"
3,dymola,Alternative,annual,"24,095.30",NaN,NaN,"8,169.10","22,210.40","10,338.00","99,021.10","875,500.14","3,502,000.56","163,833.90","1,039,334.04"


In [51]:
pue_df

{'':    Total Energy Use incl. non-IT [kWh] IT Heat Load  PUE Simulator  \
 0                           274,403.50   912,000.00 1.30    dymola   
 1                           265,765.50   912,000.00 1.29    dymola   
 2                         1,051,115.12 3,502,000.08 1.30    dymola   
 3                         1,006,615.08 3,502,000.32 1.29    dymola   
 
   Controller Design  
 0         Base-case  
 1       Alternative  
 2         Base-case  
 3       Alternative  ,
 '_sens+2':    Total Energy Use incl. non-IT [kWh] IT Heat Load  PUE Simulator  \
 0                           273,712.80   912,000.00 1.30    dymola   
 1                           262,027.90   912,000.00 1.29    dymola   
 2                         1,050,182.02 3,502,000.08 1.30    dymola   
 3                         1,001,567.88 3,502,000.32 1.29    dymola   
 
   Controller Design  
 0         Base-case  
 1       Alternative  
 2         Base-case  
 3       Alternative  ,
 '_sens-2':    Total Energy Use incl. n

## Energy performance metrics: PUE

## For the plant

In [33]:
pue_df = dict()
pue_only = dict()

for sensitivity in ['', '+2', '-2']:
    
    if sensitivity == '+2':
        sens_key = '_sens+2'
    elif sensitivity == '-2':
        sens_key = '_sens-2'
    else:
        sens_key = ''
        
    pue_df[sens_key] = PAgg_compiled[sens_key].loc[:, ["Total Energy Use incl. non-IT [kWh]", "IT Heat Load"]].copy()
    pue_only[sens_key] =pd.DataFrame()

    pue_df[sens_key]["PUE"] = pue_df[sens_key].sum(axis = 1.).divide(pue_df[sens_key]["IT Heat Load"])

    pue_only[sens_key]["PUE"] = pue_df[sens_key].sum(axis = 1.).divide(pue_df[sens_key]["IT Heat Load"])

    pue_df[sens_key]["Simulator"] = PAgg_compiled[sens_key]["Simulator"].copy()
    pue_df[sens_key]["Controller Design"] = PAgg_compiled[sens_key]["Controller Design"].copy()

    pue_only[sens_key]["Simulator"] = PAgg_compiled[sens_key]["Simulator"].copy()
    pue_only[sens_key]["Controller Design"] = PAgg_compiled[sens_key]["Controller Design"]
    # pue_df[sens_key]
    # pue_only[sens_key] = 
    # pue_df[sens_key].loc[:, "PUE", "Simulator", "Controller Design"]
    pue_only[sens_key]["Season"] = PAgg_compiled[sens_key]["Season"].copy()

pue_only

{'':    PUE Simulator Controller Design  Season
 0 1.30    dymola         Base-case  summer
 1 1.29    dymola       Alternative  summer
 2 1.30    dymola         Base-case  annual
 3 1.29    dymola       Alternative  annual,
 '_sens+2':    PUE Simulator Controller Design  Season
 0 1.30    dymola         Base-case  summer
 1 1.29    dymola       Alternative  summer
 2 1.30    dymola         Base-case  annual
 3 1.29    dymola       Alternative  annual,
 '_sens-2':    PUE Simulator Controller Design  Season
 0 1.31    dymola         Base-case  summer
 1 1.32    dymola       Alternative  summer
 2 1.30    dymola         Base-case  annual
 3 1.30    dymola       Alternative  annual}

## PUE per device (DPUE)

This metric, device-resolution PUE I define as follows:

* x - total energy use excluding the IT energy use
* y - total IT energy use (we assumed it converts to heat in the room)
* x1 - device energy use

DPUE_x1 = x1/y + x1/x

Then the sum of all devices thus matches the wiki formula for PUE = (x + y)/y

With this the stacked colors represent devices and the total bar hight the system PUE.

In [34]:
pue_df_per_device = dict()

for sensitivity in ['', '+2', '-2']:
    
    if sensitivity == '+2':
        sens_key = '_sens+2'
    elif sensitivity == '-2':
        sens_key = '_sens-2'
    else:
        sens_key = ''
        
    pue_df_per_device[sens_key] = PAgg_compiled[sens_key].loc[:, "CW Pump":"Non-IT Heat Load"].copy()
    pue_df_per_device[sens_key] = pue_df_per_device[sens_key].divide(PAgg_compiled[sens_key]["IT Heat Load"], axis = 0) + pue_df_per_device[sens_key].divide(PAgg_compiled[sens_key]["Total Energy Use incl. non-IT [kWh]"], axis = 0)
    pue_df_per_device[sens_key]["System PUE"] = pue_df_per_device[sens_key].sum(axis = 1.)

    # PAgg_compiled[sens_key][vlmap["QRooIntGaiAgg.y"]]
    pue_df_per_device[sens_key]["Simulator"] = PAgg_compiled[sens_key]["Simulator"].copy()
    pue_df_per_device[sens_key]["Controller Design"] = PAgg_compiled[sens_key]["Controller Design"]
    pue_df_per_device[sens_key]["Season"] = PAgg_compiled[sens_key]["Season"].copy()

    # remove unit from column labels
    cols = [c for c in pue_df_per_device[sens_key].columns]

    rename_dict = dict()
    for i in np.arange(len(cols)):
        rename_dict[pue_df_per_device[sens_key].columns[i]] = cols[i]

    pue_df_per_device[sens_key] = pue_df_per_device[sens_key].rename(columns = rename_dict)
    
pue_df_per_device

{'':   CW Pump WSE Pump CT Pump CHW Pump Chiller Cooling Tower Fan Supply Fan  \
 0    0.00     0.02    0.00     0.01    0.00              0.07       0.12   
 1    0.02      NaN     NaN     0.01    0.00              0.02       0.13   
 2    0.00     0.02    0.00     0.00    0.00              0.07       0.12   
 3    0.02      NaN     NaN     0.01    0.00              0.01       0.13   
 
   Non-IT Heat Load  System PUE Simulator Controller Design  Season  
 0             1.08        1.30    dymola         Base-case  summer  
 1             1.11        1.29    dymola       Alternative  summer  
 2             1.08        1.30    dymola         Base-case  annual  
 3             1.12        1.29    dymola       Alternative  annual  ,
 '_sens+2':   CW Pump WSE Pump CT Pump CHW Pump Chiller Cooling Tower Fan Supply Fan  \
 0    0.00     0.02    0.00     0.00    0.00              0.07       0.12   
 1    0.02      NaN     NaN     0.01    0.00              0.01       0.13   
 2    0.00     0

## Total energy use and PUE comparative plots

In [35]:
en_use_categories = list(vlmap.values())[:-1] + ["Non-IT Heat Load"]

df_dpue = dict()
df_en_use = dict()

for sensitivity in ['', '+2', '-2']:
    
    if sensitivity == '+2':
        sens_key = '_sens+2'
    elif sensitivity == '-2':
        sens_key = '_sens-2'
    else:
        sens_key = ''
        
    df_en_use[sens_key] = pd.melt(PAgg_compiled[sens_key], id_vars=["Simulator", "Controller Design", "Season"], value_vars = en_use_categories, var_name="Device", value_name='Energy Use [kWh]')

    device_cols = ["CW Pump", "WSE Pump", "CT Pump", "CHW Pump","Chiller","Cooling Tower Fan","Supply Fan","Non-IT Heat Load"]

    df_dpue[sens_key] = pd.melt(pue_df_per_device[sens_key], id_vars=["Simulator", "Controller Design", "Season"], value_vars = device_cols, var_name="Device", value_name='PUE')
    # df_dpue

# pue_df_per_device
df_en_use

{'':    Simulator Controller Design  Season             Device Energy Use [kWh]
 0     dymola         Base-case  summer            CW Pump            26.00
 1     dymola       Alternative  summer            CW Pump         4,583.60
 2     dymola         Base-case  annual            CW Pump            25.20
 3     dymola       Alternative  annual            CW Pump        17,210.70
 4     dymola         Base-case  summer           WSE Pump         4,003.20
 5     dymola       Alternative  summer           WSE Pump              NaN
 6     dymola         Base-case  annual           WSE Pump        15,382.00
 7     dymola       Alternative  annual           WSE Pump              NaN
 8     dymola         Base-case  summer            CT Pump           356.90
 9     dymola       Alternative  summer            CT Pump              NaN
 10    dymola         Base-case  annual            CT Pump         1,333.10
 11    dymola       Alternative  annual            CT Pump              NaN
 12    d

In [50]:
df

{'':    Simulator Controller Design  Season             Device  PUE
 0     dymola         Base-case  summer            CW Pump 0.00
 1     dymola       Alternative  summer            CW Pump 0.02
 2     dymola         Base-case  annual            CW Pump 0.00
 3     dymola       Alternative  annual            CW Pump 0.02
 4     dymola         Base-case  summer           WSE Pump 0.02
 5     dymola       Alternative  summer           WSE Pump  NaN
 6     dymola         Base-case  annual           WSE Pump 0.02
 7     dymola       Alternative  annual           WSE Pump  NaN
 8     dymola         Base-case  summer            CT Pump 0.00
 9     dymola       Alternative  summer            CT Pump  NaN
 10    dymola         Base-case  annual            CT Pump 0.00
 11    dymola       Alternative  annual            CT Pump  NaN
 12    dymola         Base-case  summer           CHW Pump 0.01
 13    dymola       Alternative  summer           CHW Pump 0.01
 14    dymola         Base-case  ann

In [56]:
#### Journal paper export

In [47]:
# data export
for key in df_en_use.keys():
    df_en_use[key].to_csv(os.path.join("appendix_data_for_paper", "energy_use" + key + ".csv"))
    

In [134]:
device_map = pd.DataFrame(data = [
    ['CHW Pump','4 CHW Pump'],
    ['CT Pump', '5 CT Pump'],
    ['CW Pump','7 CW Pump'],
    ['Chiller', '3 Chiller'],
    ['Cooling Tower Fan','2 Cooling Tower Fan'],
    ['Supply Fan','1 Supply Fan'],
    ['WSE Pump', '6 WSE Pump'],
    ['Total', '8 Total'],
], 
    columns = ['Device','enum device'])

In [135]:
device_map

,Device,enum device
0,CHW Pump,4 CHW Pump
1,CT Pump,5 CT Pump
2,CW Pump,7 CW Pump
3,Chiller,3 Chiller
4,Cooling Tower Fan,2 Cooling Tower Fan
5,Supply Fan,1 Supply Fan
6,WSE Pump,6 WSE Pump
7,Total,8 Total


In [253]:
df_en_use_export = df_en_use.copy()

df_en_use_export['_all'] = df_en_use_export['']
df_en_use_export['_all'] = df_en_use_export['_all'].drop(columns='Energy Use [kWh]')
df_en_use_export['_all']['27'] = df_en_use_export['']['Energy Use [kWh]']
df_en_use_export['_all']['29'] = df_en_use_export['_sens+2']['Energy Use [kWh]']
df_en_use_export['_all']['25'] = df_en_use_export['_sens-2']['Energy Use [kWh]']

df_en_use_export['_all']

,Simulator,Controller Design,Season,Device,27,25,29
0,dymola,Base-case,summer,CW Pump,26.00,0.70,795.50
1,dymola,Alternative,summer,CW Pump,"4,583.60","4,442.70","11,109.60"
2,dymola,Base-case,annual,CW Pump,25.20,0.00,801.40
3,dymola,Alternative,annual,CW Pump,"17,210.70","17,069.80","24,095.30"
4,dymola,Base-case,summer,WSE Pump,"4,003.20","4,003.10","3,999.00"
5,dymola,Alternative,summer,WSE Pump,NaN,NaN,NaN
6,dymola,Base-case,annual,WSE Pump,"15,382.00","15,381.80","15,377.60"
7,dymola,Alternative,annual,WSE Pump,NaN,NaN,NaN
8,dymola,Base-case,summer,CT Pump,356.90,343.60,777.40
9,dymola,Alternative,summer,CT Pump,NaN,NaN,NaN


In [254]:
df_en_use_export.keys()

dict_keys(['', '_sens+2', '_sens-2', '_all'])

In [267]:
pd.options.display.float_format = '{:,.0f}'.format

a = dict()
ta = dict()
a_export = dict()

for key in df_en_use_export.keys():
    a[key] = df_en_use_export[key].loc[
        df_en_use_export[key]['Device']!='Non-IT Heat Load'].drop(
        columns = ["Simulator"])
    
    a[key] = a[key].merge(device_map, on='Device', how='left')
    
    ta[key] = a[key].groupby(['Season', 'Controller Design']).sum().reset_index()
    
    ta[key]['Device'] = 'Total'
    
    a[key] = pd.concat([ta[key],a[key]])
    try:
        a[key] = a[key].loc[a[key]['Energy Use [kWh]']>0,:]
    except:
        a[key] = a[key].loc[a[key]['25']>0,:]
    
    a[key] = a[key].sort_values(
        ['Season','Controller Design', 'enum device','Device'])
   
    a_export[key] = a[key].groupby(['Season', 'Controller Design', 'Device'],
                                  sort=False).sum()
    a_export[key] = a_export[key].drop(columns='enum device')

In [268]:
a_export['_all']

27      25      29
Season Controller Design Device                                   
annual Alternative       Supply Fan         99,021  99,021  99,021
                         Cooling Tower Fan   6,878   2,631  10,338
                         CHW Pump            7,582   7,346   8,169
                         CW Pump            17,211  17,070  24,095
                         Total             131,115 126,068 163,834
       Base-case         Supply Fan         99,021  99,021  99,021
                         Cooling Tower Fan  56,938  56,938  56,938
                         CHW Pump            2,760   2,022   3,965
                         CT Pump             1,333   1,320   1,757
                         WSE Pump           15,382  15,382  15,378
                         Total             175,615 174,682 183,488
summer Alternative       Supply Fan         25,773  25,773  25,773
                         Cooling Tower Fan   4,834   1,886   5,438
                         Chiller               424       1  21,146
                         CHW Pump            2,151   1,926   2,570
                         CW Pump             4,584   4,443  11,110
                         Total              37,766  34,028  66,036
       Base-case         Supply Fan         25,773  25,773  25,773
                         Cooling Tower Fan  14,820  14,820  14,820
                         Chiller               156       0   5,598
                         CHW Pump            1,269     773   1,904
                         CT Pump               357     344     777
                         WSE Pump            4,003   4,003   3,999
                         CW Pump                26       1     796
                         Total              46,404  45,713  53,666

In [261]:
# help(pd.DataFrame.index)

Help on AxisProperty object:

class AxisProperty(builtins.object)
 |  Methods defined here:
 |  
 |  __delete__(self, instance, /)
 |      Delete an attribute of instance.
 |  
 |  __get__(self, instance, owner, /)
 |      Return an attribute of instance, which is of type owner.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __reduce__ = __reduce_cython__(...)
 |  
 |  __set__(self, instance, value, /)
 |      Set an attribute of instance to value.
 |  
 |  __setstate__ = __setstate_cython__(...)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  axis



In [269]:
# to latex

for key in df_en_use_export.keys():
    a_export[key].to_latex(os.path.join("appendix_data_for_paper", "energy_use_latex" + key))
    
# DataFrame.to_latex(buf=None, 
#                    columns=None, 
#                    header=True, 
#                    index=True, 
#                    na_rep='NaN', 
#                    formatters=None, 
#                    float_format=None, 
#                    sparsify=None, 
#                    index_names=True, 
#                    bold_rows=False, 
#                    column_format=None, 
#                    longtable=None, escape=None, 
#                    encoding=None, decimal='.', multicolumn=None, multicolumn_format=None, 
#                    multirow=None, caption=None, label=None, position=None)

In [232]:
a_export.keys()

dict_keys(['', '_sens+2', '_sens-2'])

In [270]:
pd.options.display.float_format = '{:,.2f}'.format

df_export = df.copy()

df_export['_all'] = df_export['']
df_export['_all'] = df_export['_all'].drop(columns='PUE')
df_export['_all']['27'] = df_export['']['PUE']
df_export['_all']['29'] = df_export['_sens+2']['PUE']
df_export['_all']['25'] = df_export['_sens-2']['PUE']

df_export['_all']

,Simulator,Controller Design,Season,Device,27,25,29
0,dymola,Base-case,summer,CW Pump,0.00,0.00,0.00
1,dymola,Alternative,summer,CW Pump,0.02,0.02,0.05
2,dymola,Base-case,annual,CW Pump,0.00,0.00,0.00
3,dymola,Alternative,annual,CW Pump,0.02,0.02,0.03
4,dymola,Base-case,summer,WSE Pump,0.02,0.02,0.02
5,dymola,Alternative,summer,WSE Pump,NaN,NaN,NaN
6,dymola,Base-case,annual,WSE Pump,0.02,0.02,0.02
7,dymola,Alternative,annual,WSE Pump,NaN,NaN,NaN
8,dymola,Base-case,summer,CT Pump,0.00,0.00,0.00
9,dymola,Alternative,summer,CT Pump,NaN,NaN,NaN


In [271]:
# sort by device case season, remove simulator, provide case/season totals

# data export
for key in df.keys():
    df[key].to_csv(os.path.join("appendix_data_for_paper", "pue" + key + ".csv"))

In [272]:
a = dict()
ta = dict()
a_export_pue = dict()

for key in df_export.keys():
    a[key] = df_export[key].drop(
        columns = ["Simulator"])
    
    a[key] = a[key].merge(device_map, on='Device', how='left')
    
    ta[key] = a[key].groupby(['Season', 'Controller Design']).sum().reset_index()
    
    ta[key]['Device'] = 'Total'
    
    a[key] = pd.concat([ta[key],a[key]])
    try:
        a[key] = a[key].loc[a[key]['PUE']>0.000001,:]
    except:
        a[key] = a[key].loc[a[key]['25']>0.000001,:]
    
    a[key] = a[key].sort_values(
        ['Season','Controller Design', 'enum device','Device'])
   
    a_export_pue[key] = a[key].groupby(['Season', 'Controller Design', 'Device'],
                                  sort=False).sum()
    
    a_export_pue[key] = a_export_pue[key].drop(columns='enum device')

In [273]:
a_export_pue['_all']
# a['']

27   25   29
Season Controller Design Device                          
annual Alternative       Supply Fan        0.13 0.13 0.12
                         Cooling Tower Fan 0.01 0.00 0.01
                         CHW Pump          0.01 0.01 0.01
                         CW Pump           0.02 0.02 0.03
                         Non-IT Heat Load  1.12 1.12 1.09
                         Total             1.29 1.29 1.30
       Base-case         Supply Fan        0.12 0.12 0.12
                         Cooling Tower Fan 0.07 0.07 0.07
                         CHW Pump          0.00 0.00 0.00
                         CT Pump           0.00 0.00 0.00
                         WSE Pump          0.02 0.02 0.02
                         Non-IT Heat Load  1.08 1.08 1.08
                         Total             1.30 1.30 1.30
summer Alternative       Supply Fan        0.13 0.13 0.12
                         Cooling Tower Fan 0.02 0.01 0.02
                         Chiller           0.00 0.00 0.10
                         CHW Pump          0.01 0.01 0.01
                         CW Pump           0.02 0.02 0.05
                         Non-IT Heat Load  1.11 1.12 1.03
                         Total             1.29 1.29 1.32
       Base-case         Supply Fan        0.12 0.12 0.12
                         Cooling Tower Fan 0.07 0.07 0.07
                         CHW Pump          0.01 0.00 0.01
                         CT Pump           0.00 0.00 0.00
                         WSE Pump          0.02 0.02 0.02
                         CW Pump           0.00 0.00 0.00
                         Non-IT Heat Load  1.08 1.08 1.06
                         Total             1.30 1.30 1.31

In [274]:
# to latex

for key in df_export.keys():
    a_export_pue[key].to_latex(os.path.join("appendix_data_for_paper", "pue_latex" + key))
    
# DataFrame.to_latex(buf=None, 
#                    columns=None, 
#                    header=True, 
#                    index=True, 
#                    na_rep='NaN', 
#                    formatters=None, 
#                    float_format=None, 
#                    sparsify=None, 
#                    index_names=True, 
#                    bold_rows=False, 
#                    column_format=None, 
#                    longtable=None, escape=None, 
#                    encoding=None, decimal='.', multicolumn=None, multicolumn_format=None, 
#                    multirow=None, caption=None, label=None, position=None)

In [ ]:
# using matplotlib, this needs update - as dataframes were turned into dictionaries with the sensitivity keys added

# from matplotlib import cm
# cmap = cm.get_cmap('Spectral')

# plot1 = PAgg_compiled[sens_key].loc[:,:vlmap['PSupFanAgg.y']].set_index(['Controller Design', 'Simulator']).plot(
#     kind='bar', x = None, stacked=True,
#     title = "Power consumption per device, controller design, and simulator, in kWh",
#     rot = 90, cmap=cmap, width=.3, grid=False).legend(loc='upper right')

# # plot1 = ax.spines["right"]
# # plot1.set_visible(False)

# fig1 = plot1.get_figure()
# fig1.set_size_inches(6, 8)
# fig1.tight_layout()
# fig1.savefig("img/energy_use.png", bbox_inches='tight')

# plot2 = pue_only.set_index(['Controller Design', 'Simulator']).plot(
#     kind='bar', x = None, stacked=False,
#     title = "PUE",
#     rot = 90, cmap=cmap, width=.3, grid=False).legend(loc='upper right')

# fig2 = plot2.get_figure()
# fig2.set_size_inches(4, 8)
# fig2.tight_layout()
# fig2.savefig("img/pue.png", bbox_inches='tight')

In [36]:
# obtain some hex colors

import plotly.express as px

print(px.colors.qualitative.Plotly)

['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']


## Energy use

In [38]:
# energy use, using plotly

from plotly.offline import iplot, init_notebook_mode

import plotly.graph_objs as go
import plotly.io as pio

font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 30,
        'color' : "#7f7f7f"}

width = 600
height = 1600

df = df_en_use.copy()

inlcude_legend = True

if inlcude_legend:
    skip_legend = ''
else:
    skip_legend = '_wo_legend'


for sensitivity in ['', '+2', '-2']:
    
    if sensitivity == '+2':
        sens_key = '_sens+2'
    elif sensitivity == '-2':
        sens_key = '_sens-2'
    else:
        sens_key = ''
        
    for sim in df[sens_key]["Simulator"].unique():

    #     title = sim.capitalize()
        title = ""

        df_s = df[sens_key][df[sens_key]["Simulator"] == sim]

        fig = go.Figure()

        fig.update_layout(
            template="simple_white",
    #         xaxis=dict(title_text="Season, Controller Design"),
    #         yaxis=dict(title_text="Energy Use [kWh]"),
            barmode="stack",
            font=dict(family=font['family'], size=font['size']),
            title=title,
            titlefont=dict(size=font['size'] * 2.0, family=font['family']),
            xaxis=dict(
                title_text="Season, Controller Design",
                titlefont=dict(
                    family=font['family'],
                    size=font['size'],
                    color=font['color'],
                ),
                tickfont=dict(size=font['size']),
            ),
            yaxis=dict(
                title_text="Energy Use [MWh]",
                range=[0,180000/1000.],
                titlefont=dict(
                    family=font['family'],
                    size=font['size'],
                    color=font['color'],
                ),
                tickfont=dict(size=font['size']),
            ),
            showlegend=inlcude_legend,
            width=width,
            height=height,
    #         margin=dict(l=margin_l, b=margin_b),
            legend=dict(
                x=0,
                y=0.9,
                bgcolor = 'rgba(255,255,255,0.4)',
                font=dict(family=font['family'], size=font['size']*1),
            ),
        )


    #     colors = ["#2A66DE", "#FFC32B", "#339933", "#cc9900", "#9933cc"]
        colors = ['#636EFA', '#37FDFC', '#A9A9A9', '#EF553B', '#00CC96', '#AB63FA', '#FECB52']

        for r, c in zip(df_s["Device"].unique(), colors):
            plot_df = df_s[df_s["Device"] == r]

            fig.add_trace(
                go.Bar(x=[plot_df["Season"].apply(
                    lambda x: x.capitalize()), plot_df["Controller Design"]], y=plot_df["Energy Use [kWh]"]/1000., 
                       name=r, marker_color=c, width = 0.5),
            )

        img_bytes = fig.to_image(format="png", engine="kaleido")

        filename = "energy_use_" + sim

        pio.write_image(fig, os.path.join("img", filename + sens_key + skip_legend + ".png"))

        pio.write_image(fig, os.path.join("img", filename + sens_key + skip_legend + ".pdf"))

        from IPython.display import Image
        Image(img_bytes)
    

In [ ]:
plot_df

### PUE device level and total PUI all-in-one

In [39]:
# data is from df_dpue

df = df_dpue.copy()

from plotly.offline import iplot, init_notebook_mode

import plotly.graph_objs as go
import plotly.io as pio

font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 30,
        'color' : "#7f7f7f"}

width = 600
height = 1600

inlcude_legend = False

if inlcude_legend:
    skip_legend = ''
else:
    skip_legend = '_wo_legend'


for sensitivity in ['', '+2', '-2']:
    
    if sensitivity == '+2':
        sens_key = '_sens+2'
    elif sensitivity == '-2':
        sens_key = '_sens-2'
    else:
        sens_key = ''

    for sim in df[sens_key]["Simulator"].unique():

    #     title = sim.capitalize()
        title = ""

        df_s = df[sens_key][df[sens_key]["Simulator"] == sim]

        fig = go.Figure()

        fig.update_layout(
            template="simple_white",
    #         xaxis=dict(title_text="Season, Controller Design"),
    #         yaxis=dict(title_text="Energy Use [kWh]"),
            barmode="stack",
            font=dict(family=font['family'], size=font['size']),
            title=title,
            titlefont=dict(size=font['size'] * 1.0, family=font['family']),
            xaxis=dict(
                title_text="Season, Controller Design",
                titlefont=dict(
                    family=font['family'],
                    size=font['size'],
                    color=font['color'],
                ),
                tickfont=dict(size=font['size']),
                ticks="",
            ),
            yaxis=dict(
                title_text="PUE",
                range=[0,1.35],
                titlefont=dict(
                    family=font['family'],
                    size=font['size'],
                    color=font['color'],
                ),
                tickfont=dict(size=font['size']),
                ticks="inside",
                nticks=20,
                showgrid=True, gridwidth=0.2, gridcolor="Gray",
            ),
            showlegend=inlcude_legend,
            width=width,
            height=height,
    #         margin=dict(l=margin_l, b=margin_b),
            legend=dict(
                x=0,
                y=0.9,
                bgcolor = 'rgba(255,255,255,0.6)',
                font=dict(family=font['family'], size=font['size']*1),
            ),
        )


    #     colors = ["#2A66DE", "#FFC32B", "#339933", "#cc9900", "#9933cc"]
        colors = ['#636EFA', '#37FDFC', '#A9A9A9', '#EF553B', '#00CC96', '#AB63FA', '#FECB52', "#FFA500"]

        for r, c in zip(df_s["Device"].unique(), colors):
            plot_df = df_s[df_s["Device"] == r]
            fig.add_trace(
                go.Bar(x=[plot_df["Season"].apply(lambda x: x.capitalize()), plot_df["Controller Design"]], 
                       y=plot_df["PUE"], name=r, marker_color=c, width = 0.5),
            )

        img_bytes = fig.to_image(format="png", engine="kaleido")

        filename = "device-level_pue_" + sim

        pio.write_image(fig, os.path.join("img", filename + sens_key + skip_legend + ".png"))

        pio.write_image(fig, os.path.join("img", filename + sens_key + skip_legend + ".pdf"))

        from IPython.display import Image
        Image(img_bytes)

### Total PUE

In [40]:
from plotly.offline import iplot, init_notebook_mode

import plotly.graph_objs as go
import plotly.io as pio


for sensitivity in ['', '+2', '-2']:
    
    if sensitivity == '+2':
        sens_key = '_sens+2'
    elif sensitivity == '-2':
        sens_key = '_sens-2'
    else:
        sens_key = ''


    font = {'family' : 'serif',
            'weight' : 'normal',
            'size'   : 16,
            'color' : "#7f7f7f"}

    width = 600
    height = 1600

    title = "Power Utilization Effectivness"


    fig = go.Figure()

    fig.update_layout(
        template="simple_white",
    #         xaxis=dict(title_text="Simulator, Controller Design"),
    #         yaxis=dict(title_text="Energy Use [kWh]"),
        barmode="group",
        font=dict(family=font['family'], size=font['size']),
        title=title,
        titlefont=dict(size=font['size'] * 2.0, family=font['family']),
        xaxis=dict(
            title_text="",
            titlefont=dict(
                family=font['family'],
                size=font['size'],
                color=font['color'],
            ),
            tickfont=dict(size=font['size']),
            ticks="inside"
        ),
        yaxis=dict(
            title_text="PUE",
            range=[1,1.4],
            titlefont=dict(
                family=font['family'],
                size=font['size'],
                color=font['color'],
            ),
            tickfont=dict(size=font['size']),
            ticks="inside"
        ),
        showlegend=True,
        width=width,
        height=height,
    #         margin=dict(l=margin_l, b=margin_b),
        legend=dict(
                x=0,
                y=1.,
            font=dict(family=font['family'], size=font['size']),
        ),
    )


    #     colors = ["#2A66DE", "#FFC32B", "#339933", "#cc9900", "#9933cc"]
    colors = ['#636EFA', '#00CC96', '#AB63FA']

    # to plot both simulators side by side use 'pue_only' directly 
    # in the zip below
    pue_single_simulator = pue_only[sens_key][pue_only[sens_key]["Simulator"] == "dymola"]
    pue_single_simulator["Controller Design Label"] = "Controller Design"

    for r, c in zip(pue_single_simulator["Season"].unique(), colors):
        plot_df = pue_single_simulator[
            pue_single_simulator["Season"] == r]
    #     fig.add_trace(
    #         go.Bar(x=[plot_df["Simulator"].apply(
    #             lambda x: x.capitalize()), plot_df["Controller Design"].apply(
    #               lambda x: x[:-8])], y=plot_df["PUE"], name=r, marker_color=c, width = 0.3),
    #     )
        fig.add_trace(
            go.Bar(x=[plot_df["Controller Design Label"], plot_df["Controller Design"]], y=plot_df["PUE"], name=r, marker_color=c, width = 0.3),
        )


    img_bytes = fig.to_image(format="png", engine="kaleido")

    filename = "pue"

    pio.write_image(fig, os.path.join("img", filename + sens_key + ".png"))

    pio.write_image(fig, os.path.join("img", filename + sens_key + ".pdf"))

    from IPython.display import Image
    Image(img_bytes)

# Create plots against an independent variable

## Example to access timeseries results for each variable

In [ ]:
# Look at some resulting variables
cases.get_list_of_case_names()
# res['optimica']['test_base'].varNames()
(t, P_agg) = res['dymola']['base_summer'].values("QRooIntGaiAgg.y")
P_agg[-1]

## Configure plots

In [ ]:
plt.rcParams['axes.facecolor']='whitesmoke'
plt.rcParams['font.size'] = 6
plt.rcParams['text.usetex'] = False
plt.rcParams['legend.facecolor'] = 'white'
plt.rcParams['legend.framealpha'] = 0.75
plt.rcParams['legend.edgecolor'] = 'none'
plt.rcParams['savefig.dpi'] = 300

def save_plot(figure, file_name):
    """ Save the figure to a pdf and png file in the directory `img`
    """
    import os
    import matplotlib.pyplot as plt
    
    out_dir = "img"
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    figure.savefig(os.path.join(out_dir, '{}.pdf'.format(file_name)),bbox_inches='tight')
    figure.savefig(os.path.join(out_dir, '{}.png'.format(file_name)),bbox_inches='tight')
    
    plt.clf()
    

def configure_axes(axes):
    """ Configure the axis style
    """
    axes.spines['right'].set_visible(False)
    axes.spines['top'].set_visible(False)
    axes.spines['left'].set_visible(False)
    axes.spines['bottom'].set_visible(False)
    axes.grid(color='lightgrey', linewidth=0.25)
    return


# ---------------------------------------------------------------------------
# helper functions and scripts

def set_cases_and_initiate_plot():
    from matplotlib.gridspec import GridSpec
    cases = ['test_base', 'test_alt']
    seasons = ['']
    num_cases = len(cases)
    num_seasons = len(seasons)
    
    fig = plt.figure(figsize=(6.5,8.))
    gs1 = GridSpec(80, 1)
    gs1.update(left=0.1, right=0.9, hspace=0.05)
    
    ax = list()
    ax.insert(0, fig.add_subplot(gs1[0:11,:]))
    ax.insert(1, fig.add_subplot(gs1[12:23,:]))
    ax.insert(2, fig.add_subplot(gs1[28:39,:]))
    ax.insert(3, fig.add_subplot(gs1[40:51,:]))
    ax.insert(4, fig.add_subplot(gs1[56:67,:]))
    ax.insert(5, fig.add_subplot(gs1[68:79,:]))
    
    # fig, ax = plt.subplots(nrows=num_cases*num_seasons, ncols=1, figsize = (6.5,8.))
    # fig, ax = plt.subplots(nrows=20, ncols=1, figsize = (6.5,8.))
    
    return cases, seasons, num_cases, num_seasons, fig, ax

def set_title(ax, title):
    left, width = .01, .97
    bottom, height = .01, .88
    right = left + width
    top = bottom + height
    
    title_str = r"$\it{" + title + "}$"
    ax.text(left, top,
            title_str,
            verticalalignment = 'center',
            horizontalalignment = 'left', 
            transform=ax.transAxes,
            fontsize = 6, color = 'k',
            bbox=dict(facecolor='white', alpha=0.75, edgecolor='none'))
    
    
def set_up_labels(i, ax, cases, seasons, num_cases, num_seasons, x_axis_label, y_axis_label):
    # Hide xtick labels and ticks on the upper case subplot (each basecase)
    if i % 2 == 0:
        hide_tick_labels(ax)

    # Print x axis title only below the lowest subplot
    if i  == num_cases*num_seasons - 1:
        ax.set_xlabel(x_axis_label)
    ax.set_ylabel(y_axis_label)
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))
     
    # Annotate case
    set_title(ax, cases[i % 2])
    # Annotate case
    # if i % 2 == 0:
    #     title_str = r"$\bf{" + seasons[i/2] + "}$" + ' (upper: ' + r"$\it{" + cases[i % 2] + "}$" + ', lower: ' + r"$\it{"  + cases[(i-1) % 2] + "}$" + ')'
    #     ax.set_title(title_str, # mg assign appropriate season/case
    #                  verticalalignment = 'top',
    #                  horizontalalignment = 'center', 
    #                  fontsize = 6, color = 'k')
        
    # Print legend only at the lower plot (g36 case)
    if i % 1 == 0:
        ax.legend(loc='center right', ncol=1)
    configure_axes(ax)
        
    plt.tight_layout(h_pad=0, v_pad = 0)
#     plt.tight_layout()
    #plt.subplots_adjust(hspace = .2)
        
def tem_conv_CtoF(T_in_degC):
    '''Converts temperature provided in degC to degF
    '''
    T_in_degF = (T_in_degC)*9./5. + 32.
    
    return T_in_degF

def tem_conv_KtoC(T_in_K):
    '''Converts temperature provided in degC to degF
    '''
    T_in_degC = T_in_K - 273.15
    
    return T_in_degC
        
def add_secondary_yaxis_for_degF(ax, time, temp_in_K):
        # Add a secondary axis with temperatures represented in F
        ax_F = ax.twinx()
        # Get limits to match with the left axis
        ax_F.set_ylim([tem_conv_CtoF(ax.get_ylim()[0]),tem_conv_CtoF(ax.get_ylim()[1])])
        # plot a "scaler" variable and make it invisible
        ax_F.plot(time, tem_conv_CtoF(temp_in_K-273.15), linewidth=0.0)
        ax_F.set_ylabel('temperature [$^\circ$F]')
        configure_axes(ax_F)
        #ax.grid(False)
        #ax.xaxis.grid()
        
def hide_tick_labels(ax):
    '''Removes labels and ticks. Kwargs: bottom controls the ticks, labelbottom the tick labels
    '''
    ax.tick_params(axis = 'x',labelbottom='off',bottom='off')

## Subplots with one set of results per subplot and many variables from each

In [ ]:
from pdb import set_trace as bp

# for a list of variables, using subplots

def plot_vs_independent_variable(
    res_set,
    var_names, 
    y_var_name_list = "weaBus.TWetBul", 
    y_legend_labels = [['$test1$', '$test2$'], ['$test3$']], 
    subplot_x_labels = ["subpl 1", "subpl 2"]):
    ''' Plots a scatter of the variables selected in var_names
    against a provided independent variable, y_var.
    
    Parameters:
    
        res_set : list or one reader object
            List of reader objects with case results, 
            index represents subplot, or one reader
            for all data
     
        var_names : list of lists
            A list of lists of variable names as provided by the reader, 
            for each subplot
            
        y_var : str
            Name of the independent variable
            
        y_legend_labels : list of lists
            A list of lists for each subplot
            
        subplot_x_labels : list
            A list of x axis labels for each subplot
    '''
    
    font = {'family' : 'serif',
            'weight' : 'normal',
            'size'   : 11}
    
    matplotlib.rc('font', **font)

    n_col = len(var_names)
        
    plt.clf()
    
    if not isinstance(res_set, list):
        res_set = np.repeat(res_set,n_col)
        
    elif not len(res_set) == n_col:
        raise Exception(
            'res_set size mismatch, it should be a list of '\
            'reader objects or a single reader object')
        

    # set the independent variable here, will be taken from the first
    # set or results
    
    y_var = dict()
    for i in range(n_col):
        (t, y_var) = res_set[i].values(y_var_name)
    
        if ".T" in y_var_name:
            y_var = tem_conv_KtoC(y_var)
    
    # subplot index
    i = 1
    
    ax = dict()
    
    fig = plt.figure(figsize=(15, 5))
    
    for subplot_var_names in var_names:
        
        v = dict()
        lbl = dict()

        j = 0
        for var in subplot_var_names:

            (t, v[var]) = res_set[i-1].values(var)
            lbl[var] = y_legend_labels[i-1][j]
            
            if ".T" in var:
                v[var] = tem_conv_KtoC(v[var])
                
            j += 1

        # create subplot
        
        ax[i] = fig.add_subplot(1, n_col, i)
        
        for key in v.keys():
            print(lbl[key])
            ax[i].scatter(v[key], y_var[i-1], label = lbl[key], 
                linewidth=0.5, s = 1.)
            
            ax[i].legend(loc='center right', ncol=1)

        hide_tick_labels(ax[i])

        ax[i].set_xlabel(subplot_x_labels[i - 1])
    
        if i == 1:
            ax[i].set_ylabel('Outdoor air wetbulb temperature [$^\circ$C]')
        
        configure_axes(ax[i])
        
        i += 1
        
        del v
        
    # customize days to display

    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

#     ax.set_xlim([min(t), min(t)+24])
#     ax.set_xticks(range(24))
 
    
    
    del y_var
    plt.autoscale()
        
    return plt

# # Create the plot for all seasons and cases
# fig = plot_vs_independent_variable(res["dymola"]["base_annual"], [["pumCHW.dp_in", "TAirSup.T"], ["pumCHW.m_flow"]], 
#                                    y_legend_labels = [["CHW pump dif. pressure", "Air supply temperature"], ["CHW pump flow"]])
# save_plot(fig, "test_many")


# fig = plot_vs_independent_variable(res["dymola"]["alt_annual"], [["TChiWatSupSen.T"], ["TConWatRetSen.T"]], 
#                                    y_legend_labels = [["CHW loop temperature [$^\circ$C]"],["CW loop temperature [$^\circ$C]"]])
# save_plot(fig, "CWreset_loop_temps")

# fig = plot_vs_independent_variable([res["dymola"]["base_annual"],res["dymola"]["alt_annual"]], 
#                                    [["waterSideEconomizerOnOff.yOn"], ["yWSEOn.y"]], 
#                                    subplot_x_labels = ["base_annual", "alt_annual"],
#                                    y_legend_labels = [["WSE on/off signal"],["WSE on/off signal"]])
# save_plot(fig, "WSE_on_vs_outdoor_wetbulb")

## One variable from each set/slice of results on one plot

In [ ]:
from pdb import set_trace as bp

# works for one variable

def plot_vs_independent_variable(
    res_set_list,
    x_var_name,
    y_var_name_list = ["weaBus.TWetBul"],
    y_legend_label_list = [''], # leave as is to not have the legend
    x_label = "x label",
    y_label = "Outdoor air wetbulb temperature [$^\circ$C]",
    x_lim = False,
    y_lim = False,
    x_ticks = False,
    y_ticks = False,
    color_list = ['#3776ab'],
    ):
    
    ''' Plots a scatter of x_var_name
    against a provided y_var_name.
    
    Parameters:
    
        res_set : list of reader objects or pandas dataframe
    '''
    n = len(res_set_list)
    
    font = {'family' : 'serif',
            'weight' : 'normal',
            'size'   : 11}
    
    matplotlib.rc('font', **font)
    
    plt.clf()
    
    # instantiate figure
    fig = plt.figure(figsize=(3.5, 3.5))
    ax = fig.add_subplot(1, 1, 1)
    
    for i in range(n):
    
        res_set = res_set_list[i]
        y_var_name = y_var_name_list[i]
        colr = color_list[i]
        if len(y_legend_label_list) == n:
            y_legend_label = y_legend_label_list[i]
        else:
            y_legend_label = ''

        if not isinstance(res_set, pd.DataFrame):
            # set an independent variable here
            (t, y_var) = res_set.values(y_var_name)
            (t, x_var) = res_set.values(x_var_name)
        else:
            x_var = np.array(res_set[x_var_name])
            y_var = np.array(res_set[y_var_name])

        if ".T" in y_var_name:
            y_var = tem_conv_KtoC(y_var)

        if ".T" in x_var_name:
            x_var = tem_conv_KtoC(x_var)
            
        # create data items on the plot
        ax.scatter(x_var, y_var, label = y_legend_label, 
            linewidth=0.5, s = 2., color = colr)

        hide_tick_labels(ax)

        ax.set_xlabel(x_label)

        ax.set_ylabel(y_label)

        # customize days to display

        if not isinstance(x_ticks, bool):
            ax.xaxis.set_ticks(x_ticks)
        if not isinstance(y_ticks, bool):
            ax.yaxis.set_ticks(y_ticks)

        if not isinstance(x_lim, bool):
            ax.set_xlim(x_lim)
        if not isinstance(y_lim, bool):
            ax.set_ylim(y_lim)

        # customize days to display

        if y_legend_label_list[0] != '':
            ax.legend(loc='lower right', ncol=1, markerscale=5.,)

        configure_axes(ax)

        del y_var, x_var
        
    return fig

# condenser water side diagrams start+++++++++++++++++++++++++
# Create plots for the report
# grab WSE-only operation (chiller off)

# base

for sensitivity in ['_plus_two', '_less_two', '']:

    if sensitivity == '_plus_two':
        sens_key = '_sens+2'
    elif sensitivity == '_less_two':
        sens_key = '_sens-2'
    else:
        sens_key = ''

    fig = plot_vs_independent_variable([data["dymola"]["base_annual"+sensitivity]],
        "condenserWater.heaPreCon.y", 
        y_var_name_list = ["val.y"],
        x_label = "Head pressure control control signal, $u_{l,ch}$",
        y_label = "Mixing valve position set-point, $y_{ch,c,m,set}$",
        x_lim = [0.,1.],
        y_lim = [0.,1.],
        x_ticks = np.arange(0, 1.2, 0.2),
        y_ticks = np.arange(0, 1.2, 0.2))

    save_plot(fig, "chiller_mixing_valve_control" + sens_key)

# alternative
for sensitivity in ['_plus_two', '_less_two', '']:

    if sensitivity == '_plus_two':
        sens_key = '_sens+2'
    elif sensitivity == '_less_two':
        sens_key = '_sens-2'
    else:
        sens_key = ''
        
    wse_only =  data["dymola"]["alt_annual"+sensitivity].loc[data["dymola"]["alt_annual"+sensitivity]["chi.on"] < 0.5,:] 
    fig = plot_vs_independent_variable([wse_only],
        "towCon.towFanSpe.fanSpeWse.wseOpe.chiWatTemCon.y", 
        y_var_name_list = ["cooTow.y"],
        x_label = "CW reset control signal, $u_{cw,pr}$",
        y_label = "Tower fan speed set-point, $r_{t,set}$",
        x_lim = [0.,1.],
        y_lim = [0.,1.],
        x_ticks = np.arange(0, 1.2, 0.2),
        y_ticks = np.arange(0, 1.2, 0.2),
        color_list = ['g'])
    save_plot(fig, "tower_fan_speed_control_wse_only" + sens_key)

    integrated =  data["dymola"]["alt_annual"+sensitivity].loc[data["dymola"]["alt_annual"+sensitivity]["chi.on"] > 0.5,:] 
    fig = plot_vs_independent_variable([integrated],
        "towCon.towFanSpe.fanSpeWse.intOpe.loaCon.y", 
        y_var_name_list = ["cooTow.y"],
        x_label = "Minimum chiller capacity control signal",
        y_label = "Tower fan speed set-point, $r_{t,set}$",
        x_lim = [0.,1.],
        y_lim = [0.,1.],
        x_ticks = np.arange(0, 1.2, 0.2),
        y_ticks = np.arange(0, 1.2, 0.2))
    save_plot(fig, "tower_fan_speed_control_integrated" + sens_key)

    fig = plot_vs_independent_variable( [data["dymola"]["alt_annual"+sensitivity]], #integrated,
        "heaPreCon.chiHeaPreLoo.conPID.y", 
        y_var_name_list = ["heaPreCon.yHeaPreConVal"],
        x_label = "Head pressure control signal, $u_{l,ch}$",
        y_label = "Modulation valve position set-point, $y_{ch,c,m,set}$",
        x_lim = [0.,1.],
        y_lim = [0.,1.],
        x_ticks = np.arange(0, 1.2, 0.2),
        y_ticks = np.arange(0, 1.2, 0.2))
    save_plot(fig, "chiller_modulation_valve_control" + sens_key)


    fig = plot_vs_independent_variable(
        [wse_only, integrated],
        "weaBus.TWetBul",
        y_var_name_list = ["cooTow.y", "cooTow.y"],
        y_legend_label_list = ['WSE-only','Integrated'], # leave as is to not have the legend
        x_label = "Outdoor air wet bulb temperature [$^\circ$C]",
        y_label = "Tower fan speed set-point, $r_{t,set}$",
        x_lim = [0.,25.],
        y_lim = [0.,1.],
        x_ticks = np.arange(0., 30., 5),
        y_ticks = np.arange(0., 1.2, 0.2),
        color_list = ['g','#3776ab'])
    save_plot(fig, "tower_fan_speed_vs_wetbulb_per_operating_mode" + sens_key)
    
# condenser water reset diagrams end+++++++++++++++++++++++++

# # Create the plot for all seasons and cases
# fig = plot_vs_independent_variable(res["dymola"]["base_annual"], "pumCHW.dp_in", "weaBus.TWetBul", x_label = "CHW loop differential pressure [Pa]")
# # fig = plot_vs_independent_variable(res["dymola"]["base_annual"], "chilledWaterReset.uChiWatPlaRes", "chilledWaterReset.TChiWatSupSet")
# save_plot(fig, "test_one_var_vs_indep")

# fig = plot_vs_independent_variable(res["dymola"]["alt_annual"], "TChiWatSupSen.T", "weaBus.TWetBul", x_label = "CHW loop temperature [$^\circ$C]")
# # fig = plot_vs_independent_variable(res["dymola"]["base_annual"], "chilledWaterReset.uChiWatPlaRes", "chilledWaterReset.TChiWatSupSet")
# save_plot(fig, "chw_temp_vs_wetbulb")

# fig = plot_vs_independent_variable(res["dymola"]["alt_annual"], "TConWatRetSen.T", "weaBus.TWetBul", x_label = "CW loop temperature [$^\circ$C]")
# # fig = plot_vs_independent_variable(res["dymola"]["base_annual"], "chilledWaterReset.uChiWatPlaRes", "chilledWaterReset.TChiWatSupSet")
# save_plot(fig, "cw_temp_vs_wetbulb")

# fig = plot_vs_independent_variable(res["dymola"]["base_annual"], "chilledWaterReset.uChiWatPlaRes", "weaBus.TWetBul", x_label = "CHW reset control signal [1]")
# # fig = plot_vs_independent_variable(res["dymola"]["base_annual"], "chilledWaterReset.uChiWatPlaRes", "chilledWaterReset.TChiWatSupSet")
# save_plot(fig, "chw_reset_u_vs_wetbulb_base")

# fig = plot_vs_independent_variable(res["dymola"]["alt_annual"], "cooTow.y", "weaBus.TWetBul", x_label = "Tower fan speed [-]")
# # fig = plot_vs_independent_variable(res["dymola"]["base_annual"], "chilledWaterReset.uChiWatPlaRes", "chilledWaterReset.TChiWatSupSet")
# save_plot(fig, "tower_fan_speed_vs_wetbulb")


In [ ]:
data["dymola"].keys()

# Control diagrams through simulation (one plot two y axis)

In [ ]:
def plot_control_diagram(
    res_set,
    y_var_name,
    sec_y_var_name,
    x_var_name = "weaBus.TWetBul",
    y_legend_label = 'y legend label',
    x_label = "x label",
    y_label = "y label",
    sec_y_label = "sec y label",
    y_lim = [0., 150000],
    sec_y_lim = [0.,25.],
    y_ticks = np.arange(0, 180000, 30000),
    sec_y_ticks = np.arange(0, 30., 5),
    ):
    
    ''' 
    '''
    
    font = {'family' : 'serif',
            'weight' : 'normal',
            'size'   : 11}
    
    matplotlib.rc('font', **font)
    
    plt.clf()
    plt.tight_layout()

    # set variables
    (t, x_var) = res_set.values(x_var_name)
    (t, y_var) = res_set.values(y_var_name)
    (t, sec_y_var) = res_set.values(sec_y_var_name)
    print(sec_y_var.mean())
    
    if ".T" in y_var_name:
        y_var = tem_conv_KtoC(y_var)
    
    if ".T" in sec_y_var_name:
        sec_y_var = tem_conv_KtoC(sec_y_var)
    print(sec_y_var.mean())
    print(x_var.mean())
    # create subplot

    fig = plt.figure(figsize=(3.5, 3.5))
    ax = fig.add_subplot(1, 1, 1)

    ax.scatter(x_var, y_var, label = y_legend_label, 
        linewidth=0.5, s = 1.)
    
    ax_2 = ax.twinx()
    # plot a "scaler" variable and make it invisible
    ax_2.scatter(x_var, sec_y_var,  
              label = y_legend_label, linewidth=0.5, s = 15., marker = 'x', color = "green")
    
    
    # use to generate chilled water reset diagrams
    ax.set_ylim(y_lim)
    ax.set_xlim([0., 1.])
    ax_2.set_ylim(sec_y_lim)
    ax_2.set_xlim([0., 1.])


    hide_tick_labels(ax)
    hide_tick_labels(ax_2)
    
    configure_axes(ax)
    configure_axes(ax_2)
    
    ax_2.set_ylabel(sec_y_label)

    ax.set_xlabel(x_label)

    ax.set_ylabel(y_label)
        
    # customize days to display

    ax.yaxis.set_ticks(y_ticks)
    ax_2.yaxis.set_ticks(sec_y_ticks)

    # ax.legend(loc='center right', ncol=1)
    
    if res_set == res["dymola"]["base_annual"]:
        ax_2.legend(['Secondary y axis'], loc='lower right', 
                    ncol=1, markerscale=2.,)
    
    del y_var, sec_y_var
        
    return plt

# chilled water reset diagrams start+++++++++++++++++++++++++
# Create plots for the report
for sensitivity in ['_plus_two', '_less_two', '']:

    if sensitivity == '_plus_two':
        sens_key = '_sens+2'
    elif sensitivity == '_less_two':
        sens_key = '_sens-2'
    else:
        sens_key = ''

    fig = plot_control_diagram(res["dymola"]["base_annual"+sensitivity], 
        "chilledWaterReset.dpChiWatPumSet", "chilledWaterReset.TChiWatSupSet", x_var_name = "chilledWaterReset.uChiWatPlaRes",
        x_label = "CHW reset control signal, $u_{chw,pr}$",
        y_label = "CHW pump DP set-point, $dp_{chw,s,set}$, [Pa]",
        sec_y_label = "CHW supply temperature set-point, $T_{chw,s,set}$, [$^\circ$C]")

    save_plot(fig, "chw_reset_control_diagram_base"+sens_key)

    fig = plot_control_diagram(res["dymola"]["alt_annual"+sensitivity], 
        "chilledWaterReset.dpChiWatPumSet", "chilledWaterReset.TChiWatSupSet", x_var_name = "chilledWaterReset.uChiWatPlaRes",
        x_label = "CHW reset control signal, $u_{chw,pr}$",
        y_label = "CHW pump DP set-point, $dp_{chw,s,set}$, [Pa]",
        sec_y_label = "CHW supply temperature set-point, $T_{chw,s,set}$, [$^\circ$C]")

    save_plot(fig, "chw_reset_control_diagram_alt"+sens_key)
    
# chilled water reset diagrams end+++++++++++++++++++++++++

# condenser water reset diagrams start+++++++++++++++++++++++++
for sensitivity in ['_plus_two', '_less_two', '']:

    if sensitivity == '_plus_two':
        sens_key = '_sens+2'
    elif sensitivity == '_less_two':
        sens_key = '_sens-2'
    else:
        sens_key = ''
        
    fig = plot_control_diagram(
        res["dymola"]["alt_annual"+sensitivity], 
        "heaPreCon.yMaxTowSpeSet", "heaPreCon.yConWatPumSpeSet", x_var_name = "heaPreCon.chiHeaPreLoo.yHeaPreCon",
        x_label = "CW reset control signal [-]",
        y_label = "Max. cooling tower fan speed [-]",
        sec_y_label = "CW pump speed [-]",
        y_lim = [0., 1.2],
        sec_y_lim = [0.,1.2],
        y_ticks = np.arange(0, 1.4, .2),
        sec_y_ticks = np.arange(0, 1.4, .2),
        )
    
    save_plot(fig, "cw_headpressure_control_diagram_alt"+sens_key)

# condenser water reset diagrams end+++++++++++++++++++++++++

# # save_plot(fig, "alt_heapreval_wse_annual_control_diagram")
# save_plot(fig, "alt_head_pressure_control_diagram")


# fig = plot_control_diagram(res["dymola"]["base_annual"], 
#     "chi.TEvaLvg", "chi.TConLvg", x_var_name = "chi.EIRFunT",
#     x_label = "Chiller energy input ratio quadratic curve",
#     y_label = "CHW Supply Temperature (Evaporator Leaving Water)",
#     sec_y_label = "CW Return Temperature (Condenser Leaving Water)")

# # save_plot(fig, "alt_heapreval_wse_annual_control_diagram")
# save_plot(fig, "base_annual_EIRFunT_vs_evaLea_and_conEnt")

# fig = plot_control_diagram(res["dymola"]["alt_annual"],
#     "towCon.towFanSpe.fanSpeWse.intOpe.loaCon.y", "towCon.towFanSpe.fanSpeWse.wseOpe.chiWatTemCon.y", x_var_name = "cooTow.y",
#     x_label = "Tower fan speed",
#     y_label = "Minimum chiller capacity control signal (Integrated)",
#     sec_y_label = "Chilled water supply temperature control loop (WSE Only)")

# save_plot(fig, "alt_tower_fan_control")

# fig = plot_control_diagram(res["dymola"]["alt_annual"], 
#     "towCon.TChiWatSup", "chilledWaterReset.TChiWatSupSet", x_var_name = "cooTow.y",
#     x_label = "Tower fan speed",
#     y_label = "CHW supply temperature [$^\circ$C]",
#     sec_y_label = "CHW supply temperature set-point [$^\circ$C]")

# save_plot(fig, "alt_tower_fan_control_1")

# fig = plot_control_diagram(res["dymola"]["alt_annual"], 
#     "cooTow.y", "pumCHW.dp", x_var_name = "weaBus.TWetBul",
#     x_label = "Outdoor wetbulb temperature [$^\circ$C]",
#     y_label = "Tower fan speed",
#     sec_y_label = "CHW dp")

# save_plot(fig, "alt_tower_fan_control_2")

In [ ]:
np.arange(0, 28, 4)

# Statistical density plots

In [ ]:
# compare base and alternative controller WSE only and integrated mode 
# as box plots vs. outdoor wetbulb temperature

plot_df = dict()

for sensitivity in ['_plus_two', '_less_two', '']:

    if sensitivity == '_plus_two':
        sens_key = '_sens+2'
    elif sensitivity == '_less_two':
        sens_key = '_sens-2'
    else:
        sens_key = ''

    base = data["dymola"]["base_annual"+sensitivity].copy()
    alt = data["dymola"]["alt_annual"+sensitivity].copy()

    base_int = base[base["chi.on"] == 1.]
    base_int = base_int[base_int["waterSideEconomizerOnOff.yOn"] == 1.]
    base_int["Design"] = "Base"
    base_int["Operating Mode"] = "Integrated"
    base_wse = base[base["chi.on"] == 0.]
    base_wse = base_wse[base_wse["waterSideEconomizerOnOff.yOn"] == 1.]
    base_wse["Design"] = "Base"
    base_wse["Operating Mode"] = "WSE-only"

    base_int["Wet Bulb Temperature, [$^\circ$C]"] = tem_conv_KtoC(base_int["weaBus.TWetBul"])
    base_int1 = base_int.loc[:, ["Operating Mode", "Design", "Wet Bulb Temperature, [$^\circ$C]"]]
    base_wse["Wet Bulb Temperature, [$^\circ$C]"] = tem_conv_KtoC(base_wse["weaBus.TWetBul"])
    base_wse1 = base_wse.loc[:, ["Operating Mode", "Design", "Wet Bulb Temperature, [$^\circ$C]"]]

    alt_int = alt[alt["chi.on"] == 1.]
    alt_int = alt_int[alt_int["yWSEOn.y"] == 1.]
    alt_int["Design"] = "Alternative"
    alt_int["Operating Mode"] = "Integrated"
    alt_wse = alt[alt["chi.on"] == 0.]
    alt_wse = alt_wse[alt_wse["yWSEOn.y"] == 1.]
    alt_wse["Design"] = "Alternative"
    alt_wse["Operating Mode"] = "WSE-only"

    alt_int["Wet Bulb Temperature, [$^\circ$C]"] = tem_conv_KtoC(alt_int["weaBus.TWetBul"])
    alt_int1 = alt_int.loc[:, ["Operating Mode", "Design", "Wet Bulb Temperature, [$^\circ$C]"]]
    alt_wse["Wet Bulb Temperature, [$^\circ$C]"] = tem_conv_KtoC(alt_wse["weaBus.TWetBul"])
    alt_wse1 = alt_wse.loc[:, ["Operating Mode", "Design", "Wet Bulb Temperature, [$^\circ$C]"]]

    plot_df[sens_key] = pd.concat((base_int1, alt_int1,base_wse1, alt_wse1))

    plot_df[sens_key]["Operating Mode"].unique()

    # Check
    print(base[base["chi.on"] == 1.].shape[0]/base.shape[0])
    print(alt[alt["chi.on"] == 1.].shape[0]/alt.shape[0])

    plot_df[sens_key].groupby(["Design", "Operating Mode"]).count()/base.shape[0]


In [ ]:
plot_df['_sens+2'][plot_df['_sens+2']["Design"] == "Alternative"]

In [ ]:
plot_df['_sens-2'][plot_df['_sens-2']["Design"] == "Alternative"]

In [ ]:
import plotly.express as px

for sensitivity in ['_plus_two', '_less_two', '',]:

    if sensitivity == '_plus_two':
        sens_key = '_sens+2'
    elif sensitivity == '_less_two':
        sens_key = '_sens-2'
    else:
        sens_key = ''

    fig = px.box(plot_df[sens_key], y="Wet Bulb Temperature, [$^\circ$C]", x="Operating Mode", color="Design", 
              hover_data=plot_df[sens_key].columns, color_discrete_sequence=['#FFD23F', '#7AC74F'])


    font = {'family' : 'serif',
            'weight' : 'normal',
            'size'   : 24,
            'color' : "#7f7f7f"}

    fig.update_layout(
        boxgap = 0.3,
        boxgroupgap = 0.5,
        template="simple_white",
    #         xaxis=dict(title_text="Season, Controller Design"),
    #         yaxis=dict(title_text="Energy Use [kWh]"),
        font=dict(family=font['family'], size=font['size']),
    #     title="Operating Mode Configuration",
        titlefont=dict(size=font['size'] * 1.0, family=font['family']),
        xaxis=dict(
            title_text="Operating mode",
            titlefont=dict(
                family=font['family'],
                size=font['size'] * 1,
                color=font['color'],
            ),
            tickfont=dict(size=font['size']*0.8),
        ),
        yaxis=dict(
            title_text="Wet bulb temperature [℃]",
            titlefont=dict(
                family=font['family'],
                size=font['size'] * 1,
                color=font['color'],
            ),
            tickfont=dict(size=font['size']*0.8),
        ),
        showlegend=sens_key == '',
        width=400,
        height=800,
    #         margin=dict(l=margin_l, b=margin_b),
        legend=dict(
            x=0.05,
            y=0.1,
            bgcolor = 'rgba(255,255,255,0.6)',
            font=dict(family=font['family'], size=font['size']*1),
        ),
    )

    fig.show()

    img_bytes = fig.to_image(format="png", engine="kaleido")

    filename = "operating_modes_vs_wetbulb_boxplots"

    pio.write_image(fig, os.path.join("img", filename + sens_key + ".png"))

    pio.write_image(fig, os.path.join("img", filename + sens_key + ".pdf"))

In [ ]:
# For those temperatures in the range 10 -15 degC, where both integrated and WSE only Operating Mode configurations exist
# describe the data

int_overlap = alt_int[(alt_int["Wet Bulb Temperature, [$^\circ$C]"] > 10)]
int_overlap = int_overlap[(int_overlap["Wet Bulb Temperature, [$^\circ$C]"] < 15)]

wse_overlap = alt_wse[(alt_wse["Wet Bulb Temperature, [$^\circ$C]"] > 10)]
wse_overlap = wse_overlap[(wse_overlap["Wet Bulb Temperature, [$^\circ$C]"] < 15)]

print("Integrated: \n\n", int_overlap.describe().round(2))

In [ ]:
print("WSE: \n\n", wse_overlap.describe().round(2))

In [ ]:
wse_overlap.head()

## Chiller performance checks

In [ ]:
# aggregate data timeseries
chi_dat = dict()
chi_dat = {"dymola": dict()}
simulators = ["dymola"]
for simulator in simulators:
    for case in ["base_annual", "base_annual_plus_two", "base_annual_less_two"]:
        chi_dat[simulator][case] = get_partial_results(
            case,  ["chi.EIRFunT", "chi.capFunT","chi.TEvaLvg", "chi.TConLvg", "chi.on", "chi.P","chi.PLR1","chi.PLR2"], 
            simulator, as_dataframe=True, equidistant = simulator == "dymola")
        chi_dat[simulator][case] = chi_dat[simulator][case].loc[(chi_dat[simulator][case].index % 2)==0,:].reset_index(drop=True)

chi = chi_dat["dymola"]["base_annual"]
chi = chi[chi.columns.drop(list(chi.filter(regex='time_')))]

chi_base_on = chi[chi["chi.on"] == 1.]
print("\n\nbase:\n\n", chi_base_on.describe())

chi = chi_dat["dymola"]["base_annual_less_two"]
chi = chi[chi.columns.drop(list(chi.filter(regex='time_')))]
chi_base_on = chi[chi["chi.on"] == 1.]
print("\n\nbase less two:\n\n", chi_base_on.describe())

chi = chi_dat["dymola"]["base_annual_plus_two"]
chi = chi[chi.columns.drop(list(chi.filter(regex='time_')))]
chi_base_on = chi[chi["chi.on"] == 1.]
print("\n\nbase plus two:\n\n", chi_base_on.describe())

In [ ]:
chi_alt_dat = dict()
chi_alt_dat = {"dymola": dict()}
simulators = ["dymola"]
for simulator in simulators:
    for case in ["alt_annual", "alt_annual_plus_two", "alt_annual_less_two"]:
        chi_dat[simulator][case] = get_partial_results(
            case,  ["chi.EIRFunT", "chi.capFunT","chi.TEvaLvg", "chi.TConLvg", "chi.on", "chi.P","chi.PLR1","chi.PLR2"], 
            simulator, as_dataframe=True, equidistant = simulator == "dymola")
        chi_dat[simulator][case] = chi_dat[simulator][case].loc[(chi_dat[simulator][case].index % 2)==0,:].reset_index(drop=True)
chi_alt = chi_dat["dymola"]["alt_annual"]

chi_alt.head()

chi_alt = chi_alt[chi_alt.columns.drop(list(chi_alt.filter(regex='time_')))]

chi_alt_on = chi_alt[chi_alt["chi.on"] == 1.]
print("\n\nalt:\n\n", chi_base_on.describe())

chi = chi_dat["dymola"]["alt_annual_less_two"]
chi = chi[chi.columns.drop(list(chi.filter(regex='time_')))]
chi_base_on = chi[chi["chi.on"] == 1.]
print("\n\nalt less two:\n\n", chi_base_on.describe())

chi = chi_dat["dymola"]["alt_annual_plus_two"]
chi = chi[chi.columns.drop(list(chi.filter(regex='time_')))]
chi_base_on = chi[chi["chi.on"] == 1.]
print("\n\nalt plus two:\n\n", chi_base_on.describe())

In [ ]:
print("alt:\n\n", chi_alt_on.describe())

In [ ]:
# For potential future investigation

check_p = chi_base_on[chi_base_on["chi.TEvaLvg"] >= chi_base_on["chi.TConLvg"]]
check_p["lift"] = -check_p["chi.TEvaLvg"].subtract(chi_base_on["chi.TConLvg"])
check_p.sort_values("lift", ascending = False)

chi_base_on

In [ ]:
check_p = chi_alt_on[chi_alt_on["chi.TEvaLvg"] <= chi_alt_on["chi.TConLvg"]]
check_p["lift"] = -check_p["chi.TEvaLvg"].subtract(chi_alt_on["chi.TConLvg"])
check_p.sort_values("lift", ascending = False)

check_p.describe()
check_p

In [ ]:
# 3d plot to check chiller performance outside of the usual range (at low lift region)

# res["dymola"]["alt_heapreval_annual"].varNames()
#  (t, sec_y_var) = res["dymola"]["alt_heapreval_annual"].values("chilledWaterReset.TChiWatSupSet")
# (t, x_var) = res["dymola"]["base_annual"].values("chi.EIRFunT")
# (t, y_var) = res["dymola"]["base_annual"].values("chi.TEvaLvg")
# (t, sec_y_var) = res["dymola"]["base_annual"].values("chi.TConEnt")
# plt.contour([y_var, sec_y_var], x_var, [10, 30, 50])

# y_var = chi_on["chi.TEvaLvg"].values
# sec_y_var = chi_on["chi.TConLvg"].values
# x_var = chi_on["chi.EIRFunT"].values

# from mpl_toolkits import mplot3d

# ax = plt.axes(projection='3d')

# sec_y_var = tem_conv_KtoC(sec_y_var)
# y_var = tem_conv_KtoC(y_var)

# fig = ax.scatter3D(y_var, sec_y_var, x_var, c=x_var,cmap='viridis', marker = '.', linewidth=0.5) 

# ax.set_xlabel('CHW Supply Temperature (Evaporator Leaving Water)')
# ax.set_ylabel('CW Return Temperature (Condenser Leaving Water)')
# ax.set_zlabel('Chiller energy input ratio quadratic curve')

# save_plot(plt, "Base - Chiller energy input ratio quadratic curve_3d")


# y_var = chi_on["chi.TEvaLvg"].values
# sec_y_var = chi_alt_on["chi.TConLvg"].values
# x_var = chi_alt_on["chi.EIRFunT"].values

# ax = plt.axes(projection='3d')

# sec_y_var = tem_conv_KtoC(sec_y_var)
# y_var = tem_conv_KtoC(y_var)

# fig = ax.scatter3D(y_var, sec_y_var, x_var, c=x_var,cmap='viridis', marker = '.', linewidth=0.5) 

# ax.set_xlabel('CHW Supply Temperature (Evaporator Leaving Water)')
# ax.set_ylabel('CW Return Temperature (Condenser Leaving Water)')
# ax.set_zlabel('Chiller energy input ratio quadratic curve')

# save_plot(plt, "alt - Chiller energy input ratio quadratic curve_3d")

# # check capFunT

# y_var = chi_alt_on["chi.TEvaLvg"].values
# sec_y_var = chi_alt_on["chi.TConLvg"].values
# x_var = chi_alt_on["chi.capFunT"].values

# ax = plt.axes(projection='3d')

# sec_y_var = tem_conv_KtoC(sec_y_var)
# y_var = tem_conv_KtoC(y_var)

# fig = ax.scatter3D(y_var, sec_y_var, x_var, c=x_var,cmap='viridis', marker = '.', linewidth=0.5) 

# ax.set_xlabel('CHW Supply Temperature (Evaporator Leaving Water)')
# ax.set_ylabel('CW Return Temperature (Condenser Leaving Water)')
# ax.set_zlabel('Cooling capacity factor function of temperature curve')

# save_plot(plt, "alt - Cooling capacity factor function of temperature curve_3d")

# y_var = chi_on["chi.TEvaLvg"].values
# sec_y_var = chi_on["chi.TConLvg"].values
# x_var = chi_on["chi.capFunT"].values

# ax = plt.axes(projection='3d')

# sec_y_var = tem_conv_KtoC(sec_y_var)
# y_var = tem_conv_KtoC(y_var)

# fig = ax.scatter3D(y_var, sec_y_var, x_var, c=x_var,cmap='viridis', marker = '.', linewidth=0.5) 

# ax.set_xlabel('CHW Supply Temperature (Evaporator Leaving Water)')
# ax.set_ylabel('CW Return Temperature (Condenser Leaving Water)')
# ax.set_zlabel('Cooling capacity factor function of temperature curve')

# save_plot(plt, "Base - Cooling capacity factor function of temperature curve_3d")

# Timeseries plots

## Chiller plant - power

*mg adjust this method to take various cases as needed

In [ ]:
def plot_power(res_set):
    ''' Main method that plots the results
    '''
    font = {'family' : 'serif',
            'weight' : 'normal',
            'size'   : 6}
    matplotlib.rc('font', **font)
    
    plt.clf()
    
    time_scale=3600.
       
    (t, pumCHW_P) = res_set.values("pumCHW.P")
    (t, pumCW_P) = res_set.values("pumCW.P")
    (t, chi_P) = res_set.values("chi.P")
    t = t/time_scale
   
    # Plot figure
    fig = plt.figure(figsize=(6.5, 2.5))
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(t, pumCHW_P, label = '$P_{chw,p}$', linewidth=0.5)
         
    # make_ticklabels_invisible(plt.gcf())
        
    # customize days to display

    ax.set_xlabel('time [h]')
    ax.set_ylabel('Chiller water pump power [W]')
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

    ax.set_xlim([min(t), min(t)+24])
    ax.set_xticks(range(24))
 
    ax.legend(loc='center right', ncol=1)

    configure_axes(ax)
        
    return plt

# # Create the plot for all seasons and cases
# fig = plot_power(res["dymola"]["base_annual"])
# save_plot(fig, "CHWpum_P")

In [ ]:
def plot_tower_fan_speed(res_set):
    ''' Main method that plots the results
    '''
    font = {'family' : 'serif',
            'weight' : 'normal',
            'size'   : 6}
    matplotlib.rc('font', **font)
    
    plt.clf()
    
    time_scale=3600.
       
    (t, twy) = res_set.values("cooTow.y")
    t = t/time_scale
   
    # Plot figure
    fig = plt.figure(figsize=(6.5, 2.5))
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(t, twy, label = '$-$', linewidth=0.5)
         
    # make_ticklabels_invisible(plt.gcf())
        
    # customize days to display

    ax.set_xlabel('time [h]')
    ax.set_ylabel('Tower fan control signal')
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

    ax.set_xlim([min(t), max(t)])
#     ax.set_xticks(range(24))
 
    ax.legend(loc='center right', ncol=1)

    configure_axes(ax)
        
    return plt

# # Create the plot for all seasons and cases
# fig = plot_tower_fan_speed(res["dymola"]["alt_annual"])
# save_plot(fig, "Tow_fan_speed_timeseries")

In [ ]:
def plot_wet_bulb(res_set):
    ''' Main method that plots the results
    '''
    font = {'family' : 'serif',
            'weight' : 'normal',
            'size'   : 6}
    matplotlib.rc('font', **font)
    
    plt.clf()
    
    time_scale=3600.
       
    (t, twy) = res_set.values("weaBus.TWetBul")
    t = t/time_scale
   
    # Plot figure
    fig = plt.figure(figsize=(6.5, 2.5))
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(t, twy, label = '$-$', linewidth=0.5)
         
    # make_ticklabels_invisible(plt.gcf())
        
    # customize days to display

    ax.set_xlabel('time [h]')
    ax.set_ylabel('Tower fan control signal')
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

    ax.set_xlim([min(t), max(t)])
#     ax.set_xticks(range(24))
 
    ax.legend(loc='center right', ncol=1)

    configure_axes(ax)
        
    return plt

# # Create the plot for all seasons and cases
# fig = plot_tower_fan_speed(res["dymola"]["alt_annual"])
# save_plot(fig, "wet_bulb_timeseries_alt")

## Outside conditions

In [ ]:
def plot_outside(res_set):
    # ------------------------------------------------------
    plt.clf()
    plt.ylim([0., 20.])
    time_scale=86400.
    # ------------------------------------------------------
    
    ax = list()
    # get a list of unique climate cases
    unique_climate_cases = list(res_set.keys())[0] # all in the same climate
    
    subplot_id = 0
    for climate in unique_climate_cases:
        (t, TOut) = res_set[case_name].values("weaBus.TDryBul")
        (t, TOutWet) = res_set[case_name].values("weaBus.TWetBul")
        
        t = t/time_scale
        
        # Generate figure and plot data
        ax.insert(subplot_id, plt.subplot(len(unique_climate_cases)*2, 1, subplot_id+1))
        
        for subplot_id in [0,2,4]:
            ax[subplot_id].plot(t, TOut-273.15, 'r', \
                   label='$T_{oa}$', linewidth=0.5)
            ax[subplot_id].set_ylabel('temperature [$^\circ$C]')
            add_secondary_yaxis_for_degF(ax[subplot_id], t, TOut)
            set_title(ax[subplot_id], '$T_{oa}$')
        
        subplot_id += 1
        
        for subplot_id in [1,3,5]:
            ax[subplot_id].plot(t, TOutWet-273.15, 'r', \
                   label='$T_{oa,w}$', linewidth=0.5)
            ax[subplot_id].set_ylabel('temperature [$^\circ$C]')
            add_secondary_yaxis_for_degF(ax[subplot_id], t, TOutWet)
            set_title(ax[subplot_id], '$T_{oa,w}$')
        
        # customize days to display
        ax[subplot_id].set_xlim([min(t)+5, min(t)+10])

        configure_axes(ax[subplot_id])
        ax[subplot_id].legend(loc='center right', ncol=1)
        
        subplot_id += 1

#     ax[5].set_xlabel('time [days]')

    return plt

# # Create the plot for all seasons and cases
# fig = plot_outside(res["dymola"])
# save_plot(fig, "outside")

cases.get_list_of_case_names()


## Timeseries snippet with both vertical axes

In [ ]:
from pdb import set_trace as bp

def plot_timeseries(
    res_set,
    y_var_name_list = ["weaBus.TWetBul", "chilledWaterReset.TChiWatSupSet", "TChiWatSupSen.T"],
    sec_y_var_name_list = ["chi.on","cooTow.y"],
    y_legend_label_list = ['y legend label', 'y legend label', 'y legend label'],
    sec_y_legend_label_list = ['sec y legend label', 'sec y legend label'],
    y_color_list = ['b','g','y'],
    sec_y_color_list = ['k','r'],
    y_linestyle = ["dotted", "dotted", "dotted"],
    sec_y_linestyle = ["dashed", "dashed"],
    y_linewidth = [.5,.5,.5],
    sec_y_linewidth = [.5,.5],
    x_label = "x axis label",
    y_label = "y axis label",
    sec_y_label = "sec y axis label",
    time_interval_s = [0.,360000.],
    time_scale = 3600.,
    y_axis_lim = [0.,35.],
    sec_y_axis_lim = [0.,1.1],
    figsize=(15, 3.5),
    fontsize=11,
    y_legend_position= [0,1.35],
    sec_y_legend_position= [1,1.35],
    show_legends = True,
    ):
    ''' 
    res_set : Reader object or dataframe
    '''
    fig = plt.figure(figsize=figsize)
    
    n_y = len(y_var_name_list)
    n_sec_y = len(sec_y_var_name_list)
    
    font = {'family' : 'serif',
            'weight' : 'normal',
            'size'   : fontsize}
    
    matplotlib.rc('font', **font)
    
    plt.clf()
    plt.tight_layout()
    
    ax = fig.add_subplot(1, 1, 1)
#     ax_2 = ax.twinx()
    
#     y_var, t_y, sec_y_var, t_sec_y = dict(), dict(), dict(), dict()
    y_legend_label, sec_y_legend_label = dict(), dict()

    # left y axis
    for i in range(n_y):

        (t_y, y_var) = res_set.values(y_var_name_list[i])
        t_y = t_y/time_scale
        
        if ".T" in y_var_name_list[i] or "TwtBulDes.y" in y_var_name_list[i] :
            y_var = tem_conv_KtoC(y_var)

        # create data items on the plot
        ax.plot(t_y, y_var, label = y_legend_label_list[i], 
            linewidth=y_linewidth[i], linestyle = y_linestyle[i],
            color = y_color_list[i])
        
        del t_y, y_var
         
    ax_2 = ax.twinx()
            
    # right y axis
    for i in range(n_sec_y):

        (t_sec_y, sec_y_var) = res_set.values(sec_y_var_name_list[i])
        t_sec_y = t_sec_y/time_scale
        
        if ".T" in sec_y_var_name_list[i] or "TwtBulDes.y" in sec_y_var_name_list[i] :
            sec_y_var = tem_conv_KtoC(sec_y_var)

        # create data items on the plot
        ax_2.plot(t_sec_y, sec_y_var, label = sec_y_legend_label_list[i], 
            linewidth=sec_y_linewidth[i], linestyle = sec_y_linestyle[i],
            color = sec_y_color_list[i])
        
        del t_sec_y, sec_y_var
        
    ax.set_xlim([
        time_interval_s[0]/time_scale, 
        time_interval_s[1]/time_scale])
    
    ax_2.set_xlim([
        time_interval_s[0]/time_scale, 
        time_interval_s[1]/time_scale])

    ax.set_ylim(y_axis_lim)
    ax_2.set_ylim(sec_y_axis_lim)

    hide_tick_labels(ax)
    hide_tick_labels(ax_2)
    
    configure_axes(ax)
    configure_axes(ax_2)
    
    if show_legends:
        
        y_legend = ax.legend(loc='upper left', ncol=1, markerscale=3.,bbox_to_anchor = y_legend_position,facecolor=(1, 1, 1, 0.1))
        sec_y_legend = ax_2.legend(loc='upper right', ncol=1, markerscale=2.,bbox_to_anchor = sec_y_legend_position,facecolor=(1, 1, 1, 0.9))
        
        y_legend.get_frame().set_alpha(None)
        y_legend.get_frame().set_facecolor((1, 1, 1, 0.))
        sec_y_legend.get_frame().set_alpha(None)
        sec_y_legend.get_frame().set_facecolor((1, 1, 1, 0.))
        
    
    ax_2.set_ylabel(sec_y_label)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)

    return plt



In [ ]:
# Chiller ON comparative evaluation plots

# with modulation and mixing vale positions and chiller PLRs

# fig = plot_timeseries(
#     res["dymola"]["base_annual"],
#     y_var_name_list = ["weaBus.TWetBul", "chilledWaterReset.TChiWatSupSet", "TCHWEntChi.T", "TAirSup.T"],
#     sec_y_var_name_list = ["chi.on","chillerOnOff.chiSwi.hysteresis.u","chillerOnOff.chiSwi.hysteresis.uHigh",'val.y','chi.PLR1'],
#     y_legend_label_list = ["Wet bulb temperature, $T_{oa,w}$, [$^\circ$C]", "CHW supply temperature set-point, $T_{chw,s,set}$, [$^\circ$C]", 'CHW temperature downstream of WSE, $T_{chw,r,d}$, [$^\circ$C]', "Zone supply air temperature, $T_{z,s}$, [$^\circ$C]"],
#     sec_y_legend_label_list = ['Chiller enable signal [-]', 'Temperature difference between $T_{chw,r,d}$ and $T_{chw,s,set}$, [$K$]', 'Chiller status enable hysteresis deadband, [$K$]','CW mixing valve position [-]','Chiller PLR (design minimum is 0.1)'],
#     y_linestyle = ["dashed", "dotted", "dashed", "dashed"],
#     sec_y_linestyle = ["dotted", '-.', '-.',"dotted","dashed"],
#     x_label = "Time [h of the year]",
#     y_color_list = ['#6CC551','#6699CC','#D62246','#0F8B8D'],
#     sec_y_color_list = ['#011627','#B14AED','#FF8C42','k','r'],
#     y_linewidth = [1,3,1,2],
#     sec_y_linewidth = [2,1,1,1,0.5],
#     y_label = "Temperature [$^\circ$C]",
#     sec_y_label = "Status [-] or temperature difference [K]",
#     time_interval_s = [16002000.,16092000.],
#     time_scale = 3600.,
#     y_axis_lim = [0.,30.],
#     sec_y_axis_lim = [-0,6.],
#     y_legend_position= [-0.07,1.6],
#     sec_y_legend_position= [1.11,1.6],
#     figsize=(14, 3.5),
#     fontsize=14,
#     )

# save_plot(fig, "chiller_enable_disable_condition_timesnap_base"+sens_key)

# plt.close()

# fig = plot_timeseries(
#     res["dymola"]["alt_annual"],
#     y_var_name_list = ["weaBus.TWetBul", "chilledWaterReset.TChiWatSupSet", "TChiWatSupSen.T", "staSetCon.TWsePre","TAirSup.T"],
#     sec_y_var_name_list = ["chi.on","staSetCon.staUp.hysTSup.u","staSetCon.staUp.hysTSup.uHigh","staSetCon.staDow.hysTSup.u","staSetCon.staDow.hysTSup.uHigh",'val5.y','chi.PLR1'],
#     y_legend_label_list = ["Wet bulb temperature, $T_{oa,w}$, [$^\circ$C]", "CHW supply temperature set-point, $T_{chw,s,set}$, [$^\circ$C]", 'CHW supply temperature, $T_{chw,s}$, [$^\circ$C]', "Downstream of WSE prediction, $T_{chw,r,d,p}$, [$^\circ$C]","Zone supply air temperature, $T_{z,s}$, [$^\circ$C]"],
#     sec_y_legend_label_list = ['Chiller enable signal [-]', 'Temperature difference between $T_{chw,s}$ and $T_{chw,s,set}$, [$K$]', 'Chiller status enable hysteresis deadband, [$K$]', 'Temperature difference between $T_{chw,r,d,p}$ and $T_{chw,s,set}$, [$K$]', 'Chiller status disable hysteresis deadband, [$K$]','CW modulation valve position [-]','Chiller PLR (minimum lift is 5K)'],
#     y_linestyle = ["dashed", "dotted", "dashed",'-.', "dashed"],
#     sec_y_linestyle = ["dotted", '-.', '-.',"-.","-.","dotted","dashed"],
#     x_label = "Time [h of the year]",
#     y_color_list = ['#6CC551','#6699CC','#D62246', '#7D7C84','#0F8B8D'],
#     sec_y_color_list = ['#011627','#B14AED','#FF8C42','#874000','#DBD56E', 'k', 'r'],
#     y_linewidth = [1,3,1,1,2],
#     sec_y_linewidth = [2,1,1,1,1,1,0.5],
#     y_label = "Temperature [$^\circ$C]",
#     sec_y_label = "Status [-] or temperature difference [K]",
#     time_interval_s = [16002000.,16092000.],
#     time_scale = 3600.,
#     y_axis_lim = [0.,30.],
#     sec_y_axis_lim = [-0,6.],
#     y_legend_position= [-0.07,1.85],
#     sec_y_legend_position= [1.11,1.85],
#     figsize=(14, 3.5),
#     fontsize=14,
#     )

# save_plot(fig, "chiller_enable_disable_condition_timesnap_alternative"+sens_key)

# plt.close()

# without modulation and mixing vale positions and chiller PLRs, to avoid duplication

for sensitivity in ['_plus_two', '_less_two', '',]:

    if sensitivity == '_plus_two':
        sens_key = '_sens+2'
    elif sensitivity == '_less_two':
        sens_key = '_sens-2'
    else:
        sens_key = ''

    fig = plot_timeseries(
        res["dymola"]["base_annual"+sensitivity],
        y_var_name_list = ["weaBus.TWetBul", "chilledWaterReset.TChiWatSupSet", "TCHWEntChi.T", "TAirSup.T"],
        sec_y_var_name_list = ["chi.on","chillerOnOff.chiSwi.hysteresis.u","chillerOnOff.chiSwi.hysteresis.uHigh"],
        y_legend_label_list = ["Wet bulb temperature, $T_{oa,w}$, [$^\circ$C]", "CHW supply temperature set-point, $T_{chw,s,set}$, [$^\circ$C]", 'CHW temperature downstream of WSE, $T_{chw,r,d}$, [$^\circ$C]', "Zone supply air temperature, $T_{z,s}$, [$^\circ$C]"],
        sec_y_legend_label_list = ['Chiller enable signal [-]', 'Temperature difference between $T_{chw,r,d}$ and $T_{chw,s,set}$, [$K$]', 'Chiller status enable hysteresis deadband, [$K$]'],
        y_linestyle = ["dashed", "dotted", "dashed", "dashed"],
        sec_y_linestyle = ["dotted", '-.', '-.'],
        x_label = "Time [h of the year]",
        y_color_list = ['#6CC551','#6699CC','#D62246','#0F8B8D'],
        sec_y_color_list = ['#011627','#B14AED','#FF8C42'],
        y_linewidth = [1.5,3,1,2],
        sec_y_linewidth = [2,1,2],
        y_label = "Temperature [$^\circ$C]",
        sec_y_label = "Status [-] or temperature difference [K]",
        time_interval_s = [16002000.,16092000.],
        time_scale = 3600.,
        y_axis_lim = [0.,30.],
        sec_y_axis_lim = [-0,6.],
        y_legend_position= [-0.07,1.5],
        sec_y_legend_position= [1.11,1.5],
        figsize=(14, 4),
        fontsize=14,
        )

    save_plot(fig, "chiller_enable_disable_condition_timesnap_base"+sens_key)

    plt.close()

    fig = plot_timeseries(
        res["dymola"]["alt_annual"+sensitivity],
        y_var_name_list = ["weaBus.TWetBul", "chilledWaterReset.TChiWatSupSet", "TChiWatSupSen.T", "staSetCon.TWsePre","TAirSup.T"],
        sec_y_var_name_list = ["chi.on","staSetCon.staUp.hysTSup.u","staSetCon.staUp.hysTSup.uHigh","staSetCon.staDow.hysTSup.u","staSetCon.staDow.hysTSup.uHigh"],
        y_legend_label_list = ["Wet bulb temperature, $T_{oa,w}$, [$^\circ$C]", "CHW supply temperature set-point, $T_{chw,s,set}$, [$^\circ$C]", 'CHW supply temperature, $T_{chw,s}$, [$^\circ$C]', "Downstream of WSE prediction, $T_{chw,r,d,p}$, [$^\circ$C]","Zone supply air temperature, $T_{z,s}$, [$^\circ$C]"],
        sec_y_legend_label_list = ['Chiller enable signal [-]', 'Temperature difference between $T_{chw,s}$ and $T_{chw,s,set}$, [$K$]', 'Chiller status enable hysteresis deadband, [$K$]', 'Temperature difference between $T_{chw,r,d,p}$ and $T_{chw,s,set}$, [$K$]', 'Chiller status disable hysteresis deadband, [$K$]'],
        y_linestyle = ["dashed", "dotted", "dashed",'-.', "dashed"],
        sec_y_linestyle = ["dotted", '-.', "solid", '-.', "solid"],
        x_label = "Time [h of the year]",
        y_color_list = ['#6CC551','#6699CC','#D62246', '#7D7C84','#0F8B8D'],
        sec_y_color_list = ['#011627','#B14AED','#FF8C42','#874000','#DBD56E'],
        y_linewidth = [1.5,3,1,0.5,2],
        sec_y_linewidth = [2,2,2,1,1],
        y_label = "Temperature [$^\circ$C]",
        sec_y_label = "Status [-] or temperature difference [K]",
        time_interval_s = [16002000.,16092000.],
        time_scale = 3600.,
        y_axis_lim = [0.,30.],
        sec_y_axis_lim = [-0,6.],
        y_legend_position= [-0.07,1.6],
        sec_y_legend_position= [1.11,1.6],
        figsize=(14, 4),
        fontsize=14,
        )

    save_plot(fig, "chiller_enable_disable_condition_timesnap_alternative"+sens_key)

    plt.close()

In [ ]:
# near-stationary wet bulb ramp for system WSE-only applicability determination

fig = plot_timeseries(
    res["dymola"]["base_wse_limit"],
    y_var_name_list = ["TwtBulDes.y", "TAirSup.T"],
    sec_y_var_name_list = ["chi.on"],
    y_legend_label_list = ["Wet bulb temperature near-stationary ramp, $T_{oa,w}$", "Zone supply air temperature, $T_{z,s}$, [$^\circ$C]"],
    sec_y_legend_label_list = ['Chiller enable signal [-]'],
    y_linestyle = ["solid", "dashed"],
    sec_y_linestyle = ["dotted"],
    x_label = "Time [h]",
    y_color_list = ['#6CC551','#0F8B8D'],
    sec_y_color_list = ['#011627'],
    y_linewidth = [1,.5],
    sec_y_linewidth = [2,],
    y_label = "Temperature [$^\circ$C]",
    sec_y_label = "Status [-]",
    time_interval_s = [0.,300000.],
    time_scale = 3600.,
    y_axis_lim = [0.,30.],
    sec_y_axis_lim = [-0,1.2],
    figsize = (4,4),
    fontsize = 14,
    y_legend_position= [-0.2,1.40],
    sec_y_legend_position= [0.84,1.2]
    )

save_plot(fig, "wse_limit_test_base"+sens_key)

plt.close()

fig = plot_timeseries(
    res["dymola"]["alt_wse_limit"],
    y_var_name_list = ["TwtBulDes.y", "TAirSup.T"],
    sec_y_var_name_list = ["chi.on"],
    y_legend_label_list = ["Wet bulb temperature near-stationary ramp, $T_{oa,w}$", "Zone supply air temperature, $T_{z,s}$, [$^\circ$C]"],
    sec_y_legend_label_list = ['Chiller enable signal [-]'],
    y_linestyle = ["solid", "dashed"],
    sec_y_linestyle = ["dotted"],
    x_label = "Time [h]",
    y_color_list = ['#6CC551','#0F8B8D'],
    sec_y_color_list = ['#011627'],
    y_linewidth = [1,.5],
    sec_y_linewidth = [2,],
    y_label = "Temperature [$^\circ$C]",
    sec_y_label = "Status [-]",
    time_interval_s = [0.,300000.],
    time_scale = 3600.,
    y_axis_lim = [0.,30.],
    sec_y_axis_lim = [-0,1.2],
    figsize = (4,4),
    fontsize = 14,
    y_legend_position= [0,1.15],
    sec_y_legend_position= [1,1.35],
    show_legends = False
    )

save_plot(fig, "wse_limit_test_alternative"+sens_key)

plt.close()

fig = plot_timeseries(
    res["dymola"]["base_below_wse_limit"],
    y_var_name_list = ["TwtBulDes.y", "TAirSup.T"],
    sec_y_var_name_list = ["chi.on"],
    y_legend_label_list = ["Wet bulb temperature near-stationary ramp, $T_{oa,w}$", "Zone supply air temperature, $T_{z,s}$, [$^\circ$C]"],
    sec_y_legend_label_list = ['Chiller enable signal [-]'],
    y_linestyle = ["solid", "dashed"],
    sec_y_linestyle = ["dotted"],
    x_label = "Time [h]",
    y_color_list = ['#6CC551','#0F8B8D'],
    sec_y_color_list = ['#011627'],
    y_linewidth = [1,.5],
    sec_y_linewidth = [2,],
    y_label = "Temperature [$^\circ$C]",
    sec_y_label = "Status [-]",
    time_interval_s = [0.,300000.],
    time_scale = 3600.,
    y_axis_lim = [0.,30.],
    sec_y_axis_lim = [-0,1.2],
    figsize = (4,4),
    fontsize = 14,
    y_legend_position= [-0.2,1.30],
    sec_y_legend_position= [1,1.05],
    show_legends = False
    )

save_plot(fig, "wse_below_limit_base"+sens_key)

plt.close()

fig = plot_timeseries(
    res["dymola"]["alt_below_wse_limit"],
    y_var_name_list = ["TwtBulDes.y", "TAirSup.T"],
    sec_y_var_name_list = ["chi.on"],
    y_legend_label_list = ["Wet bulb temperature near-stationary ramp, $T_{oa,w}$", "Zone supply air temperature, $T_{z,s}$, [$^\circ$C]"],
    sec_y_legend_label_list = ['Chiller enable signal [-]'],
    y_linestyle = ["solid", "dashed"],
    sec_y_linestyle = ["dotted"],
    x_label = "Time [h]",
    y_color_list = ['#6CC551','#0F8B8D'],
    sec_y_color_list = ['#011627'],
    y_linewidth = [1,.5],
    sec_y_linewidth = [2,],
    y_label = "Temperature [$^\circ$C]",
    sec_y_label = "Status [-]",
    time_interval_s = [0.,300000.],
    time_scale = 3600.,
    y_axis_lim = [0.,30.],
    sec_y_axis_lim = [0,1.2],
    figsize = (4,4),
    fontsize = 14,
    y_legend_position= [0,1.35],
    sec_y_legend_position= [1,1.35],
    show_legends = False
    )

save_plot(fig, "wse_below_limit_alternative"+sens_key)

plt.close()